# Load pre-trained UMBC vectors

In [2]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

In [5]:
w2v = 'embeddings/1A_en_UMBC_tokenized.vectors.txt'
model = KeyedVectors.load_word2vec_format(w2v, binary=False)
# pre-compute L2 norms of vectors\\\\
model.init_sims(replace=True)

In [ ]:
len(model.index2word)

# embeddings size is 219523 terms

In [54]:
model.most_similar(u'florida', topn=20)

[(u'sarasota', 0.7943328619003296),
 (u'dade', 0.7730554342269897),
 (u'alabama', 0.7612541913986206),
 (u'gainesville', 0.7478644847869873),
 (u'louisiana', 0.738013505935669),
 (u'miami', 0.7371755242347717),
 (u'carolina', 0.7347331047058105),
 (u'north_carolina', 0.7287717461585999),
 (u'pensacola', 0.7225453853607178),
 (u'maryland', 0.7108623385429382),
 (u'texas', 0.7063707113265991),
 (u'lauderdale', 0.7022767663002014),
 (u'ohio', 0.6949695944786072),
 (u'new_jersey', 0.6765778660774231),
 (u'virginia', 0.6628003120422363),
 (u'tennessee', 0.6603942513465881),
 (u'california', 0.6600708961486816),
 (u'georgia', 0.6539464592933655),
 (u'missouri', 0.6458120942115784),
 (u'hillsborough', 0.6452142000198364)]

In [ ]:
# no word is capitalised in model vocab
assert len(list(filter(lambda k: k.istitle(), model.vocab.keys()))) == 0


## Load training, test data

In [91]:
#import codecs
import io
import os
import unicodecsv as csv
from collections import defaultdict

def read_subsumptions(filenames):
    hypo, hyper = filenames
        
    data_list, gold_list, subsumptions = [], [], []
    
    # load data items
    with open(hypo, mode='r') as f:        
        reader = csv.reader(f, delimiter='\t', quoting=csv.QUOTE_NONE, encoding='utf-8')
        for row in reader:
            data_list.append(row[0])
            
    with io.open(hyper, mode= 'r') as f:        
        reader = csv.reader(f, delimiter='\t', quoting=csv.QUOTE_NONE, encoding='utf-8')
        for row in reader:            
            gold_list.append(row)
      
    # make sure we have the same number of elements in each list
    assert len(data_list) == len(gold_list)
    
    for data_item, gold_terms in zip(data_list, gold_list):
        for gold_item in gold_terms:
            data_item = data_item.replace(" ", "_").lower()
            gold_item = gold_item.replace(" ", "_").lower()
            subsumptions.append((data_item, gold_item))
    
    return subsumptions

def read_vocab(filename):
        
    vocab = []    
    # load data items
    with open(filename, mode='r') as f:        
        reader = csv.reader(f, delimiter='\t', quoting=csv.QUOTE_NONE, encoding='utf-8')
        for row in reader:
            vocab_item = row[0].replace(" ", "_").lower()
            vocab.append(vocab_item)
                              
    return vocab


In [92]:
data_file_names = list(map(lambda x: './data/shared_task/1A.english.%s.data.txt'%(x), ['trial', 'test', 'training']))
gold_file_names = list(map(lambda x: './data/shared_task/1A.english.%s.gold.txt'%(x), ['trial', 'test', 'training']))
vocab_file_name = './data/shared_task/1A.english.vocabulary.txt'

file_names = zip(data_file_names, gold_file_names)
# 0 = trial; 1 = test; 2 = train
valid_subs = read_subsumptions(file_names[0])
test_subs = read_subsumptions(file_names[1])
train_subs = read_subsumptions(file_names[2])
vocabulary = read_vocab(vocab_file_name)


In [ ]:
# ensure that model vocab is greater than given vocab
assert len(model.index2word) >= len(vocabulary)

In [93]:
# eliminate training tuples for which no embedding exists
from collections import Counter

def get_terms_having_vectors(dataset):        
    query, hyper = \
    zip(*[(q,h) for q, h in dataset 
          if q in model and h in model])
    
    return list(query), list(hyper)


train_query, train_hyper = get_terms_having_vectors(train_subs)
test_query, test_hyper = get_terms_having_vectors(test_subs)
valid_query, valid_hyper = get_terms_having_vectors(valid_subs)
vocab = list(filter(lambda w: w in model, vocabulary))

assert len(train_query) == len(train_hyper)
assert len(test_query) == len(test_hyper)
assert len(valid_query) == len(valid_hyper)

In [100]:
from collections import Counter

# person is the most popular hypernym in training set (310 times), followed by city (63)
hypernym_distrib = Counter(train_hyper + test_hyper + valid_hyper)
for v, k in  sorted(((value, key) for (key,value) in hypernym_distrib.items()), reverse = True):
    print (k,v)

# There are 4,233 unique hypernyms in all
print len(hypernym_distrib)
print sum(hypernym_distrib.values())

(u'person', 612)
(u'city', 98)
(u'writer', 77)
(u'leader', 73)
(u'state', 70)
(u'work_of_art', 67)
(u'animal', 65)
(u'constructed_structure', 64)
(u'plant', 61)
(u'natural_phenomenon', 60)
(u'software', 59)
(u'film', 57)
(u'town', 56)
(u'phenomenon', 55)
(u'software_program', 54)
(u'computer_program', 52)
(u'software_package', 51)
(u'company', 51)
(u'picture', 50)
(u'politician', 49)
(u'technical_specification', 48)
(u'movie', 45)
(u'movement', 45)
(u'country', 45)
(u'physical_phenomenon', 43)
(u'locale', 42)
(u'sportsperson', 40)
(u'show', 40)
(u'moving_picture', 39)
(u'moving-picture_show', 38)
(u'mechanism', 37)
(u'instrument', 37)
(u'computer_software', 37)
(u'transportation', 36)
(u'transport', 36)
(u'intellectual', 36)
(u'enterprise', 36)
(u'chief', 36)
(u'sport', 35)
(u'public_building', 35)
(u'medium', 35)
(u'computer_programme', 35)
(u'athlete', 35)
(u'picture_show', 34)
(u'move', 34)
(u'motion_picture', 34)
(u'motion', 33)
(u'communication_medium', 33)
(u'worker', 32)
(u'spec

(u'excretion', 2)
(u'exclusive_right', 2)
(u'exchange', 2)
(u'evildoing', 2)
(u'european_nation', 2)
(u'eristic', 2)
(u'equipment_failure', 2)
(u'equilibrium', 2)
(u'environmental_pollution', 2)
(u'environmental_condition', 2)
(u'entryway', 2)
(u'entranceway', 2)
(u'enrollee', 2)
(u'enlargement', 2)
(u'emotionality', 2)
(u'emotionalism', 2)
(u'electromagnetic_radiation', 2)
(u'electricity', 2)
(u'electrical_conductance', 2)
(u'effort', 2)
(u'edible_nut', 2)
(u'eater', 2)
(u'earthquake', 2)
(u'drug_of_abuse', 2)
(u'dose', 2)
(u'domesticated_animal', 2)
(u'domestic_dog', 2)
(u'domestic_animal', 2)
(u'dog_type', 2)
(u'dog_breed', 2)
(u'documentary', 2)
(u'doctrine', 2)
(u'doc', 2)
(u'disturbance', 2)
(u'display', 2)
(u'dislike', 2)
(u'dishware', 2)
(u'disgust', 2)
(u'diseased_person', 2)
(u'discussion', 2)
(u'discrimination', 2)
(u'disbursement', 2)
(u'disbursal', 2)
(u'dirtying', 2)
(u'dirtiness', 2)
(u'diplomat', 2)
(u'diocese', 2)
(u'dimensional_analysis', 2)
(u'dimension', 2)
(u'dicta

(u'optical_microscope', 1)
(u'optical_disk', 1)
(u'opposition', 1)
(u'oppositeness', 1)
(u'opportunity', 1)
(u'opponent', 1)
(u'operating_system_distribution', 1)
(u'online_database', 1)
(u'on-line_database', 1)
(u'olympiad', 1)
(u'olive', 1)
(u'older_adult', 1)
(u'oilseed', 1)
(u'oil', 1)
(u'official_residence', 1)
(u'official_document', 1)
(u'office_suite', 1)
(u'office_furniture', 1)
(u'offering', 1)
(u'offer', 1)
(u'odonate', 1)
(u'octopod', 1)
(u'ocean', 1)
(u'occurrent', 1)
(u'obstruent', 1)
(u'observer', 1)
(u'observatory', 1)
(u'observation', 1)
(u'obliteration', 1)
(u'object-orientation', 1)
(u'oak_wood', 1)
(u'nymph', 1)
(u'numerical_quantity', 1)
(u'nucleus', 1)
(u'nucleic_acid_sequence', 1)
(u'nucleic_acid', 1)
(u'nuclear_physicist', 1)
(u'nozzle', 1)
(u'novel_adaptation', 1)
(u'not-for-profit', 1)
(u'north_american_indian', 1)
(u'nonverbal_communication', 1)
(u'nonvascular_plant', 1)
(u'nonsmoker', 1)
(u'nonsense', 1)
(u'nonprofit_organization', 1)
(u'nonprofit', 1)
(u'non

(u'blood_circulation', 1)
(u'blood', 1)
(u'blister', 1)
(u'bleb', 1)
(u'blanket', 1)
(u'blank_space', 1)
(u'bladder', 1)
(u'black_person', 1)
(u'black_people', 1)
(u'black_magic', 1)
(u'black_belt', 1)
(u'black', 1)
(u'bishopric', 1)
(u'biscuit', 1)
(u'biotic_community', 1)
(u'biosystem', 1)
(u'biomedical_scientist', 1)
(u'biomaterial', 1)
(u'biological_weapon', 1)
(u'biological_sequence', 1)
(u'biological_research', 1)
(u'biological_membrane', 1)
(u'biography', 1)
(u'biographical_film', 1)
(u'biofilm', 1)
(u'biocide', 1)
(u'biochemist', 1)
(u'binomen', 1)
(u'binary_operation', 1)
(u'binary_arithmetic_operation', 1)
(u'billards', 1)
(u'bibliography', 1)
(u'bibliographic_index', 1)
(u'bibliographic_database', 1)
(u'bevel_gear', 1)
(u'beryl', 1)
(u'benevolence', 1)
(u'benefit', 1)
(u'bend', 1)
(u'belonging', 1)
(u'bell_shape', 1)
(u'bell', 1)
(u'believer', 1)
(u'beholder', 1)
(u'bedding', 1)
(u'bed_clothing', 1)
(u'beauty_treatment', 1)
(u'beautification', 1)
(u'beard', 1)
(u'beam', 1)
(

## Construct synonyms for training, testing and validation terms

In [3]:
from collections import defaultdict

def get_synyonyms(hyponyms, hypernyms, n=15):
    synonyms = {}
    
    # prepare hypernym lookup dictionary
    hyper_lookup = defaultdict(list)
    for q, h in zip(hyponyms, hypernyms):
        hyper_lookup[q].append(h)
                
    for term in set(hyponyms):        
        synonyms[term] = list(filter(lambda x: x not in hyper_lookup[x], zip(*model.most_similar(term, topn=20))[0]))[:n]
        
    return synonyms
    
#get_synyonyms(train_query + test_query + valid_query, train_hyper + test_hyper + valid_hyper)    
#get_synyonyms(valid_query, valid_hyper)    

def get_random(hyponyms, hypernyms, vocab, n = 15):
    random_words = {}
    
    # prepare hypernym lookup dictionary
    hyper_lookup = defaultdict(list)
    for q, h in zip(hyponyms, hypernyms):
        hyper_lookup[q].append(h)
            
    for term in set(hyponyms):                
        some_words = np.random.choice(vocab, 20, replace=False)        
        random_words[term] = list(filter(lambda x: x not in hyper_lookup[x], some_words))[:n]
            
    return random_words



In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Data class that encapsulates all word-based data I need to train the various algorithms
# We assume that we have all pre-filtered any words that don't feature in the embeddings
class Data:
    def __init__(self, 
                 train_query, train_hyper, 
                 test_query, test_hyper, 
                 valid_query, valid_hyper, 
                 vocab, embeddings):
        
                
        # encapsulate input variables so that all the data can be passed via class instance reference
        self.train_query = train_query
        self.train_hyper = train_hyper
        self.test_query = test_query
        self.test_hyper = test_hyper
        self.valid_query = valid_query
        self.valid_hyper = valid_hyper
        self.vocab = vocab
        
        #self.synonyms = synonyms
                
        # determine dimensionality of embeddings
        self.embeddings_dim = embeddings['animal'].shape[0]
        
        print ("Tokenising words...")
        # intialise and fit tokenizer
        self.tokenizer = tokenizer = Tokenizer(num_words = 300000, filters='')
        self.tokenizer.fit_on_texts(train_query + test_query + valid_query + vocab)
        
        print ("Creating embedding matrix...")
        # construct embedding_matrix
        self.embedding_matrix = np.zeros((len(self.tokenizer.word_index)+1, self.embeddings_dim), dtype='float32')

        for word, i in self.tokenizer.word_index.items():
            if i < len(self.tokenizer.word_index) + 1:
                embedding_vector = embeddings[word]
                if embedding_vector is not None:
                    # normalise vector (already normalised)
                    #embedding_vector /= np.linalg.norm(embedding_vector)
                    self.embedding_matrix[i,:] = embedding_vector  
        # confirm shape
        assert self.embedding_matrix.shape == (len(self.tokenizer.word_index)+1, self.embeddings_dim)
        
        print ("Creating random words/synonyms...")
        self.random_words = get_random(train_query + test_query + valid_query, train_hyper + test_hyper + valid_hyper, vocab)  
        self.synonyms = get_synyonyms(train_query + test_query + valid_query, train_hyper + test_hyper + valid_hyper)

In [5]:
#data = Data(train_query, train_hyper, test_query, test_hyper, valid_query, valid_hyper, vocab, model)
import pickle
import os

dest = os.path.join('.', 'pickle')
#pickle.dump(data, open(os.path.join(dest, 'semeval_data.pkl'), 'wb'), protocol=2)
data = pickle.load(open(os.path.join(dest, 'semeval_data.pkl'), 'rb'))


In [4]:
print (data.random_words['bomber'])
print (data.synonyms['bomber'])


[u'subvention', u'noblesse_oblige', u'odometer', u'sore_throat', u'harries', u'congener', u'yobo', u'marquet', u'pleuromamma', u'm\xe9tis', u'emperor_of_rome', u'confliction', u'zerbo', u'semicomatose', u'elutriator']
[u'fighter', u'bomb', u'fighter_plane', u'cruise_missile', u'fighter_aircraft', u'anti-aircraft', u'attack_aircraft', u'shoot_down', u'heavy_bomber', u'air_force', u'airliner', u'air_attack', u'air_defense', u'surface-to-air_missile', u'fighter-bomber']


In [6]:
# function that returns negative samples alongside set of positive samples
# we need to pass:
# the batch hyponym terms, batch of hypernym terms, negative_tuples, tokenizer 
# to create sequences
def extend_batch_with_negatives(batch_X_term, batch_X_hyper, negative_tuples,                              
                                tokenizer, m):
    # initialise negative tuples container
    positive_words = [tokenizer.index_word[term_id] for term_id in batch_X_term.flatten()]
    
    # tokenize -ve samples
    neg_terms, neg_hyper = [], []
    for n in positive_words:
        for n2 in negative_tuples[n][:m]:
            neg_terms.append(n)
            neg_hyper.append(n2)
    
    neg_terms_seq = tokenizer.texts_to_sequences(neg_terms)
    neg_hyper_seq = tokenizer.texts_to_sequences(neg_hyper)

    # before increasing size of our batch, let's set the actual y values
    # the first n terms are true (1s), and the rest are the -ve samples (0)
    batch_y_label = np.concatenate((
            np.ones(batch_X_term.shape[0]),
            np.zeros(len(neg_terms_seq))
    ))
    # finally, stack -ve sequences at the bottom of +ves to 
    # create our final training batch
    # at most, batch size will be 192 samples            

    batch_X_term = np.vstack((batch_X_term, np.array(neg_terms_seq)))
    batch_X_hyper = np.vstack((batch_X_hyper, np.array(neg_hyper_seq)))
    
    return batch_X_term, batch_X_hyper, batch_y_label

# Evaluation

In [7]:
def convert_hypernyms_to_one_line(dataset):
    ordered_queries = sorted(list(set(dataset[0])))
    one_line = {}
    for w in ordered_queries:
        word_hypernyms = [h for q, h in zip(*dataset) if q == w]
        one_line[w] = word_hypernyms
    return one_line

In [8]:
# taken from task_scorer.py provided with shared task resources
def mean_reciprocal_rank(r):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    r = np.asarray(r).nonzero()[0]
    return 1. / (r[0] + 1) if r.size else 0.

def precision_at_k(r, k, n):
    """Score is precision @ k
    Relevance is binary (nonzero is relevant).
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Precision @ k
    Raises:
        ValueError: len(r) must be >= k
    """
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return (np.mean(r)*k)/min(k,n)
    # Modified from the first version. Now the gold elements are taken into account

def average_precision(r,n):
    """Score is average precision (area under PR curve)
    Relevance is binary (nonzero is relevant).
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1, n) for k in range(r.size)]
    #Modified from the first version (removed "if r[k]"). All elements (zero and nonzero) are taken into account
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(r,n):
    """Score is mean average precision
    Relevance is binary (nonzero is relevant).
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean average precision
    """
    return average_precision(r,n)



In [9]:
# predictions is a dictionary whereby key is query term and value is a list of ranked hypernym predictions
def get_evaluation_scores(dataset, predictions):
    all_scores = []    
    scores_names = ['MRR', 'MAP', 'P@1', 'P@5', 'P@10']
    for query, gold_hyps in convert_hypernyms_to_one_line(dataset).items():

        avg_pat1 = []
        avg_pat2 = []
        avg_pat3 = []

        pred_hyps = predictions[query]
        gold_hyps_n = len(gold_hyps)    
        r = [0 for i in range(15)]

        for j in range(len(pred_hyps)):
            if j < gold_hyps_n:
                pred_hyp = pred_hyps[j]
                if pred_hyp in gold_hyps:
                    r[j] = 1

        avg_pat1.append(precision_at_k(r,1,gold_hyps_n))
        avg_pat2.append(precision_at_k(r,5,gold_hyps_n))
        avg_pat3.append(precision_at_k(r,10,gold_hyps_n))    

        mrr_score_numb = mean_reciprocal_rank(r)
        map_score_numb = mean_average_precision(r,gold_hyps_n)
        avg_pat1_numb = sum(avg_pat1)/len(avg_pat1)
        avg_pat2_numb = sum(avg_pat2)/len(avg_pat2)
        avg_pat3_numb = sum(avg_pat3)/len(avg_pat3)

        score_results = [mrr_score_numb, map_score_numb, avg_pat1_numb, avg_pat2_numb, avg_pat3_numb]
        all_scores.append(score_results)
    return scores_names, all_scores


## Helper functions to run evaluation routine

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# alternative hypernym generator by applying Phi weights to hyponym and see which 
# words are closest to this vector
def crim_get_hypernym(word, tokenizer, phi, cluster_weight, bias, embeddings, top):
    
    q_idx = tokenizer.word_index[word]
    q = embeddings[q_idx] 

    projections = np.dot(q, phi)
    #projections /= np.linalg.norm(projections, axis=1).reshape(-1,1)
    
    sim_matrix = np.dot(cluster_weight.T, np.dot(embeddings[1:], projections.T).T) + bias
    top_ranked_sequence = np.argsort(sim_matrix[0])[::-1][:top]
    
    return zip(tokenizer.sequences_to_texts(top_ranked_sequence.reshape(-1,1) + 1), 
               sim_matrix.flatten()[top_ranked_sequence])

In [11]:
def crim_max_get_hypernym(word, tokenizer, phi, cluster_weight, bias, embeddings, top):
    q_idx = tokenizer.word_index[word]
    q = embeddings[q_idx] 

    projections = np.dot(q, phi)
    
    projections /= np.linalg.norm(projections, axis=1).reshape(-1,1)
    
    sim_matrix = np.dot(embeddings[1:], projections.T)    
    max_sim = np.max(sim_matrix, 1).reshape(1,-1)
    
    sim_matrix = np.dot(cluster_weight.T, max_sim) + bias
    top_ranked_sequence = np.argsort(sim_matrix[0])[::-1][:top]
    
    return zip(tokenizer.sequences_to_texts(top_ranked_sequence.reshape(-1,1) + 1), 
               sim_matrix.flatten()[top_ranked_sequence])
    
    
def crim_mean_get_hypernym(word, tokenizer, phi, cluster_weight, bias, embeddings, top):
    q_idx = tokenizer.word_index[word]
    q = embeddings[q_idx] 

    projections = np.dot(q, phi)
    
    projections /= np.linalg.norm(projections, axis=1).reshape(-1,1)
    
    sim_matrix = np.dot(embeddings[1:], projections.T)    
    mean_sim = np.mean(sim_matrix, 1).reshape(1,-1)
    
    sim_matrix = np.dot(cluster_weight.T, mean_sim) + bias
    top_ranked_sequence = np.argsort(sim_matrix[0])[::-1][:top]
    
    return zip(tokenizer.sequences_to_texts(top_ranked_sequence.reshape(-1,1) + 1), 
               sim_matrix.flatten()[top_ranked_sequence])


In [12]:
def cluster_get_hypernym(word, tokenizer, phi, cluster_weight, bias, embeddings, top):
    q_idx = tokenizer.word_index[word]
    q = embeddings[q_idx] 
    
    projections = np.dot(q, phi)
    s = np.dot(embeddings[1:], projections.T)
    linear_combination = (s.T * cluster_weight) + bias

    best_projection = np.max(linear_combination, axis=0)
    top_words = np.argsort(best_projection)[::-1][:top]
    
    return zip(tokenizer.sequences_to_texts(top_words.reshape(-1,1) + 1), 
               best_projection[top_words])

    

In [13]:
# function which generates top 15 predictions for each hyponym query term
# and returns results as dictionary
def predict_crim_hypernyms(queries, tokenizer, model, algol):
    #hyper_candidates = [[data.tokenizer.word_index[hyper]] for hyper in data.tokenizer.word_index.keys()]
    #hyper_candidates = [[data.tokenizer.word_index[hyper]] for hyper in set(data.valid_hyper)]
    
    ordered_queries = sorted(list(set(queries)))
    results = {}
        
    # extract the Phi matrices out of trained model
    dense = [l.get_weights()[0] for l in model.layers if type(l) == Dense and l.name.startswith('Phi') ]
    dense = np.asarray(dense)
    
    # extract affine transform layer weights
    cluster_weight = model.get_layer(name='Prediction').get_weights()[0]
    bias = model.get_layer(name='Prediction').get_weights()[1]
    #embeddings = model.get_layer(name="TermEmbedding").get_weights()[0]
    embeddings = [l for l in model.layers if type(l) == Model][0].get_layer(name='TermEmbedding').get_weights()[0]

    
    for idx, word in enumerate(ordered_queries):        
        if (idx + 1) % 100 == 0:
            print ("Done", idx + 1)
        
        #predicted_hypers = alt_get_hypernym(word, model, data, dense, 15)
        predicted_hypers = algol(word, tokenizer, dense, cluster_weight, bias, embeddings, 15)
        results[word] = [h for h, p in predicted_hypers]
        
    return results


In [14]:
def predict_cluster_hypernyms(queries, tokenizer, cluster_list, knn_model=None):
    #hyper_candidates = [[data.tokenizer.word_index[hyper]] for hyper in data.tokenizer.word_index.keys()]
    #hyper_candidates = [[data.tokenizer.word_index[hyper]] for hyper in set(data.valid_hyper)]
    
    ordered_queries = sorted(list(set(queries)))
    
    results = {}
        
    # embeddings are present in a "shared" model that is used as the first layer of each cluster
    embeddings = [l for l in cluster_list[0].model.layers if type(l) == Model][0].get_layer(name='TermEmbedding').get_weights()[0]
    
    # extract the Phi matrices out of trained model
    dense = np.zeros((len(cluster_list), embeddings.shape[1], embeddings.shape[1]))
    lr_weights = np.zeros((len(cluster_list), 1))
    lr_bias = np.zeros((len(cluster_list), 1))

    for idx, cluster in enumerate(cluster_list):
        dense[idx] = cluster.model.get_layer(name='Phi0').get_weights()[0]
        lr_weights[idx] = cluster.model.get_layer(name='Prediction').get_weights()[0]
        lr_bias[idx] = cluster.model.get_layer(name='Prediction').get_weights()[1]    
    
    for idx, word in enumerate(ordered_queries):        
        if (idx + 1) % 100 == 0:
            print ("Done", idx + 1)
            
            
        if knn_model:            
            cluster_probs = knn_model.predict_proba(embeddings[tokenizer.word_index[word]].reshape(1,-1))
            cluster_idx = np.where(cluster_probs > 0.)[1]                        
            predicted_hypers = cluster_get_hypernym(word, tokenizer, 
                                                    dense[cluster_idx], 
                                                    lr_weights[cluster_idx], 
                                                    lr_bias[cluster_idx], embeddings, 15)
        else:        
            #predicted_hypers = alt_get_hypernym(word, model, data, dense, 15)
            predicted_hypers = cluster_get_hypernym(word, tokenizer, 
                                                    dense, lr_weights, lr_bias, 
                                                    embeddings, 15)
        
        results[word] = [h for h, p in predicted_hypers]
        
    return results

In [113]:
predict_cluster_hypernyms(['cold'], data.tokenizer, clusters)


{'cold': [u'illness',
  u'disease',
  u'weather',
  u'life-threatening',
  u'mild',
  u'cold',
  u'common_cold',
  u'weather_condition',
  u'cold_weather',
  u'severe',
  u'frostbite',
  u'respiratory_infection',
  u'hot_weather',
  u'fever',
  u'contagious']}

In [15]:
# This method retrieves the words which our model considers the most probably hypernyms.  
# Problem with this method is that it's excruciatingly slow so I developed a numpy-based
# algorithm which is significantly faster.  Refer to alt_get_hypernym

def crim_get_top_hypernyms(query, hyper_candidates, model, data, top):    
    candidates = data.tokenizer.texts_to_sequences(data.vocab)
    candidates = np.asarray(candidates).flatten()
    
    query_id = data.tokenizer.word_index[query]
    predictions = model.predict([np.asarray([query_id] * len(data.vocab)), candidates])

    best_predictions = np.argsort(predictions.flatten())[::-1][:top]
    return list(map(lambda x: (data.vocab[x], predictions[x][0]), best_predictions))
    

In [111]:
crim_get_top_hypernyms('downfall', None, crim_model, data, 15)

[(u'mass_media', 0.9961901),
 (u'judgement', 0.9954072),
 (u'value_system', 0.9939751),
 (u'judgment', 0.9937524),
 (u'demean', 0.9928692),
 (u'good_person', 0.9926629),
 (u'oneself', 0.99262685),
 (u'exercising', 0.992411),
 (u'condone', 0.9912377),
 (u'person', 0.9911773),
 (u'omission', 0.99114054),
 (u'excusing', 0.9909513),
 (u'public_servant', 0.99080867),
 (u'acknowledge', 0.9900271),
 (u'truthful', 0.98968166)]

In [112]:
predict_crim_hypernyms(['downfall'], data.tokenizer, crim_model, crim_get_hypernym)

{'downfall': [u'mass_media',
  u'judgement',
  u'value_system',
  u'judgment',
  u'demean',
  u'good_person',
  u'oneself',
  u'exercising',
  u'condone',
  u'person',
  u'omission',
  u'excusing',
  u'public_servant',
  u'acknowledge',
  u'truthful']}

In [ ]:
candidates = data.tokenizer.texts_to_sequences(data.vocab)
candidates = np.asarray(candidates).flatten()

predictions = np.zeros((5, len(data.vocab)))
query_id = data.tokenizer.word_index['starcraft']
for idx, c in enumerate(cluster_list):
    predictions[idx] = c.model.predict([np.asarray([query_id] * len(data.vocab)), candidates]).flatten()
    
best_projection = np.max(predictions, axis=0)
top_words = np.argsort(best_projection)[::-1][:15]

zip( map ( lambda x: data.tokenizer.index_word[candidates[x]], top_words), 
               best_projection[top_words])


# Positive down-sampling

In [80]:
from collections import Counter
hypernym_distrib = Counter(data.train_hyper)

hypernym_prob = {}

for v, k in  sorted(((value, key) for (key,value) in hypernym_distrib.items()), reverse = True):
    #print k, v, np.round(np.sqrt(1./v), 2)
    hypernym_prob[k] = np.round(np.sqrt(1./v), 2)
    
hypernym_prob

{u'change_of_location': 1.0,
 u'electrical_element': 1.0,
 u'dimensional_analysis': 1.0,
 u'extracurricular': 1.0,
 u'dogness': 0.71,
 u'yellow': 1.0,
 u'narcotic': 1.0,
 u'obstruction': 0.38,
 u'protest': 1.0,
 u'circuitry': 0.45,
 u'lighter-than-air_craft': 1.0,
 u'mansion': 1.0,
 u'captain': 1.0,
 u'hate': 0.71,
 u'u.s._state': 0.38,
 u'looking': 1.0,
 u'orthopaedics': 1.0,
 u'particulate_matter': 1.0,
 u'assessment': 1.0,
 u'electricity': 0.71,
 u'head_of_government': 0.71,
 u'television_program': 0.24,
 u'photographic_printing': 1.0,
 u'bludgeon': 1.0,
 u'lyrist': 0.58,
 u'saccharide': 1.0,
 u'archaeologist': 1.0,
 u'artificial_language': 1.0,
 u'medical_doctor': 0.71,
 u'honorific': 0.41,
 u'political_economy': 0.58,
 u'herbicide': 1.0,
 u'primary_color': 0.58,
 u'merchant': 0.5,
 u'ordnance': 1.0,
 u'digit': 1.0,
 u'hormone': 0.71,
 u'moiety': 1.0,
 u'risk': 0.58,
 u'family_name': 0.38,
 u'social_control': 0.29,
 u'penchant': 1.0,
 u'applied_scientist': 0.71,
 u'retail_outlet': 

In [82]:
#[np.random.random() for _ in range(100)]
subsamples = []
dropped = []
for q, h in zip(data.train_query, data.train_hyper):
    if np.random.random() < hypernym_prob[h]:
        subsamples.append((q,h))
    else:
        dropped.append((q,h))

In [100]:
downsampled_hyper = Counter(zip(*dropped)[1])
print len(subsamples), len(dropped)
print len(zip(data.train_query, data.train_hyper))
for v, k in  sorted(((value, key) for (key,value) in downsampled_hyper.items()), reverse = True):
    print k, v

6134 5645
11779
person 297
city 53
leader 37
work_of_art 36
picture 33
writer 32
natural_phenomenon 32
animal 32
locale 28
film 28
computer_software 28
constructed_structure 27
technical_specification 26
plant 26
movie 26
state 25
software 25
moving_picture 25
moving-picture_show 25
software_program 24
computer_code 24
sport 23
show 23
motion_picture 23
software_package 22
picture_show 22
movement 22
country 22
computer_programme 22
travel 21
computer_program 21
company 21
transportation 20
transmission_channel 20
town 20
physical_phenomenon 20
musical_work 20
function 20
channel 20
transport 19
public_building 19
phenomenon 19
move 19
coding_system 19
code 19
social_event 18
sickness 18
piece_of_land 18
communication_medium 18
chief 18
traveling 17
specifications 17
disease 17
boss 17
travelling 16
television_program 16
storage_medium 16
politician 16
pic 16
mechanism 16
figure 16
competitor 16
tv_program 15
television_show 15
technology 15
political_leader 15
piece_of_work 15
passeng

sex_organ 2
set_theory 2
serial 2
senior_management 2
seaman 2
sea 2
scientific_instrument 2
scholarly_person 2
scholar 2
satellite 2
sailing_vessel 2
sailing_ship 2
sailing_boat 2
sailboat 2
sail 2
rock_'n'_roll 2
river_transport 2
risk 2
rhetorical_device 2
retail_store 2
remembering 2
religious_ceremony 2
reference 2
reason 2
ratio 2
railroad_track 2
railroad_station 2
quantitative_relation 2
putting_to_death 2
public_university 2
prophet 2
programing_language 2
program_line 2
profits 2
prof 2
primary_color 2
price 2
predisposition 2
precipitation 2
portraying 2
portrayal 2
point_of_reference 2
poet 2
plug-in 2
playwright 2
pike 2
pig 2
pictorial_representation 2
photograph 2
philosophy_of_language 2
pebble 2
payment 2
patrician 2
path 2
parcelling 2
painting 2
organ 2
operator 2
offense 2
offence 2
note 2
net 2
necromancy 2
natural_language 2
native_plant 2
narrative_mode 2
musical_instrument 2
municipal_corporation 2
mountain_range 2
money 2
misconduct 2
microhabitat 2
methodology

# Keras Projection Learning Models

In [ ]:
def get_simple_class_model(phi_k=1, train_embeddings=False,\
                   embeddings_dim=300, vocab_size=1000,\
                   embeddings_matrix=None,
                   phi_init = None,
                   phi_activity_regularisation = None,
                   sigmoid_kernel_regularisation = None,
                   sigmoid_bias_regularisation = None,
                   sigmoid_kernel_constraint = None,
                   dropout_rate = 0.,
                   learning_rate = 0.001
                  ):
    
    hypo_input  = Input(shape=(1,), name='Hyponym')
    hyper_input = Input(shape=(1,), name='Hypernym')
    
    embedding_layer = Embedding(vocab_size + 1, embeddings_dim, embeddings_constraint = UnitNorm(axis=1), 
                                input_length=1, name='TermEmbedding')
    
    
    hypo_embedding = embedding_layer(hypo_input)    
    hyper_embedding = embedding_layer(hyper_input)
    
    # Add Dropout to avoid overfit    
    hypo_embedding = Dropout(dropout_rate, name='Dropout_Hypo')(hypo_embedding) 
        
    phi =  Dense(embeddings_dim, activation=None, use_bias=False, 
                 activity_regularizer=phi_activity_regularisation,
                 kernel_initializer=phi_init,
                 name='Phi') (hypo_embedding)
            
    phi = Flatten()(phi)            
    hyper_embedding = Flatten()(hyper_embedding)    
    
    word_concat = concatenate([phi, hyper_embedding], axis=-1, name='Concat')        
    word_concat = Dropout(dropout_rate)(word_concat)
                                
    predictions = Dense(1, activation="sigmoid", name='Prediction',
                        use_bias=True,
                        kernel_initializer='glorot_normal',                        
                        kernel_constraint= sigmoid_kernel_constraint,
                        bias_initializer='glorot_normal',                                                
                        kernel_regularizer=sigmoid_kernel_regularisation,
                        bias_regularizer=sigmoid_bias_regularisation
                       ) (word_concat)

    # instantiate model
    model = Model(inputs=[hypo_input, hyper_input], outputs=predictions)
        
    # inject pre-trained embedding weights into Embedding layer
    model.get_layer(name='TermEmbedding').set_weights([embeddings_matrix])
    model.get_layer(name='TermEmbedding').trainable = train_embeddings    

    adam = Adam(lr = learning_rate, beta_1 = 0.9, beta_2 = 0.9, clipnorm=1.)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
        
    return model

In [15]:
from tensorflow.keras import backend as K
from tensorflow.keras.constraints import Constraint

class ForceToOne (Constraint):    
    def __call__(self, w):
        w /= w
        return w

In [122]:
from tensorflow.keras.layers import Input, Dense, Embedding, Dot, Flatten, Concatenate, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal, Zeros, Ones
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.keras.constraints import UnitNorm, MinMaxNorm
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import backend as K
import tensorflow as tf

def get_embeddings_model(embedding_matrix):
    hypo_input = Input(shape=(1,))
    hyper_input = Input(shape=(1,))

    word_embedding = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], name='TermEmbedding',
                               embeddings_constraint = UnitNorm(axis=1))

    hypo_embedding = word_embedding(hypo_input)
    hyper_embedding = word_embedding(hyper_input)

    embedding_model = Model(inputs=[hypo_input, hyper_input], outputs=[hypo_embedding, hyper_embedding])

    # inject pre-trained embeddings into this mini, resusable model/layer
    embedding_model.get_layer(name='TermEmbedding').set_weights([embedding_matrix])
    embedding_model.get_layer(name='TermEmbedding').trainable = False
    
    return embedding_model

def custom_loss(hypo_phi_tensor):    
    def inner_product(y_true, y_pred):                
        simil = 0.01 * K.mean(hypo_phi_tensor ** 2)        
        return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1) + simil                        
        
    return inner_product

def get_CRIM_model(phi_k=1, 
                   train_embeddings = False,
                   embeddings_dim = 300,
                   #, vocab_size=1000,
                   #embeddings_matrix=None,
                   embeddings_layer = None,
                   phi_init = None,
                   phi_activity_regularisation = None,
                   sigmoid_kernel_regularisation = None,
                   sigmoid_bias_regularisation = None,
                   sigmoid_kernel_constraint = None,
                   dropout_rate = 0.,
                   learning_rate = 0.001,
                   normalised = True,
                   max_or_combine = True # if true then max
                  ):
    
    hypo_input  = Input(shape=(1,), name='Hyponym')
    hyper_input = Input(shape=(1,), name='Hypernym')
    
    #embedding_layer = Embedding(vocab_size + 1, embeddings_dim, embeddings_constraint = UnitNorm(axis=1), 
    #                            input_length=1, name='TermEmbedding')        
    
    hypo_embedding, hyper_embedding = embeddings_layer([hypo_input, hyper_input])
    #hypo_embedding = embeddings_layer(hypo_input)    
    #hyper_embedding = embeddings_layer(hyper_input)
    
    # Add Dropout to avoid overfit    
    hypo_embedding = Dropout(dropout_rate, name='Dropout_Hypo')(hypo_embedding)
    hyper_embedding = Dropout(dropout_rate, name='Dropout_Hyper')(hyper_embedding)
    
    phi_layer = []
    for i in range(phi_k):
        phi_layer.append(Dense(embeddings_dim, activation=None, use_bias=False, 
                               activity_regularizer=phi_activity_regularisation,
                               kernel_initializer=phi_init,                               
                               name='Phi%d' % (i)) (hypo_embedding))
        

    #phi1 = Dense(embeddings_dim, activation=None, use_bias=False, 
                #kernel_initializer=random_identity, name='Phi1')(hypo_embedding)

    if phi_k == 1:
        # flatten tensors
        phi = Flatten(name='Flatten_Phi')(phi_layer[0])
        hyper_embedding = Flatten(name='Flatten_Hyper')(hyper_embedding)    
    else:
        phi = concatenate(phi_layer, axis=1)
    
    phi = Dropout(dropout_rate, name='Dropout_Phi')(phi)
    
    # this is referred to as "s" in the "CRIM" paper    
    phi_hyper = Dot(axes=-1, normalize=normalised, name='DotProduct1')([phi, hyper_embedding])
    
    if (phi_k == 1):
        phi_hypo = Dot(axes=-1, normalize=False, name='DotProduct2')([phi, hypo_embedding])
        regul_loss = custom_loss(phi_hypo)
            
    
    if phi_k > 1:
        if max_or_combine:
            phi_hyper = Lambda(lambda x: K.max(x, axis=1, keepdims=True))(phi_hyper)
        phi_hyper = Flatten(name='Flatten_PhiHyper')(phi_hyper)
    
    predictions = Dense(1, activation="sigmoid", name='Prediction',
                        use_bias=True,
                        #kernel_initializer=Zeros,
                        kernel_initializer='random_normal',
                        kernel_constraint= sigmoid_kernel_constraint,                        
                        bias_initializer=Zeros(),                        
                        #bias_initializer='random_normal',
                        kernel_regularizer=sigmoid_kernel_regularisation,                        
                        bias_regularizer=sigmoid_bias_regularisation
                       ) (phi_hyper)

    # instantiate model
    model = Model(inputs=[hypo_input, hyper_input], outputs=predictions)
        
    # inject pre-trained embedding weights into Embedding layer
    #model.get_layer(name='TermEmbedding').set_weights([embeddings_matrix])
    
    embeddings = [l for l in model.layers if type(l) == Model][0].get_layer(name='TermEmbedding').trainable = train_embeddings
    #model.get_layer(name='TermEmbedding').trainable = train_embeddings    

    adam = Adam(lr = learning_rate, beta_1 = 0.9, beta_2 = 0.9, clipnorm=1.)
    if phi_k == 1:
        model.compile(optimizer=adam, loss=regul_loss, metrics=['accuracy'])
    else:
        model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [18]:
from tensorflow.keras import models
from tensorflow.keras.utils import get_custom_objects
from tensorflow.python.framework import dtypes
from tensorflow.keras.initializers import Initializer

class CustomInitializer:
    def __call__(self, shape, dtype='float32', partition_info=None):
        ci = CustomIdentity()
        return ci.random_identity(shape, dtype, partition_info)    

class RandomIdentity(Initializer):
    def __init__(self, dtype=dtypes.float32):
        self.dtype = dtypes.as_dtype(dtype)

    
    def __call__(self, shape, dtype=None, partition_info=None):
        if dtype is None:
            dtype = self.dtype
        
        rnorm = K.random_normal((shape[-1],shape[-1]), mean=0., stddev=0.01)        
        #identity = K.eye(shape[-1], dtype='float32')        
        rident = tf.eye(shape[-1]) * rnorm
        return rident
    
    def get_config(self):
        return {"dtype": self.dtype.name}
    
class RandomPlusIdentity(Initializer):
    def __init__(self, dtype=dtypes.float32):
        self.dtype = dtypes.as_dtype(dtype)

    
    def __call__(self, shape, dtype=None, partition_info=None):
        if dtype is None:
            dtype = self.dtype
        
        rnorm = K.random_normal((shape[-1],shape[-1]), mean=0., stddev=0.01)    
        rident = tf.eye(shape[-1]) + rnorm
        return rident            
    
    def get_config(self):
        return {"dtype": self.dtype.name}
        

get_custom_objects().update({'RandomIdentity': RandomIdentity})
get_custom_objects().update({'RandomPlusIdentity': RandomPlusIdentity})


# idea is to train an initial model to generate reasonabl projection matrices that
# when applied to a hyponym, gets it close to the hypernym in question.

# after initial training, we will inject the phi and sigmoid layer trained weights into the model;
# the projections will be frozen but instead we will adjust the embeddings.  
# prediction layer weights may also be modified 
def get_CRIM_model_freeze_phi(model,
                              train_phi=False, 
                              train_embeddings = True,
                              train_sigmoid = True,                              
                              sigmoid_kernel_regularisation = None,
                              sigmoid_bias_regularisation = None,
                              sigmoid_kernel_constraint = None,
                              dropout_rate = 0.,
                              learning_rate = 0.0005):
    
    # retain original model; create copy of passed model
    new_model = models.clone_model(model)    
    # retain model but transfer learned weights
    # get embeddings weights
    #embeddings = model.get_layer(name='TermEmbedding').get_weights()[0]
    
    # get embeddings layer which is "embedded" in a sub-model in model
    embeddings = [l for l in model.layers if type(l) == Model][0].get_layer(name='TermEmbedding').get_weights()[0]
    # get projection matrices
    dense = map(lambda x: x.get_weights()[0], [l for l in model.layers if l.name.startswith('Phi')])
    dense = np.asarray(dense)
    # get sigmoid weights
    lr_weights = model.get_layer(name='Prediction').get_weights()
    
    # inject pre-trained embedding weights into Embedding layer
    #new_model.get_layer(name='TermEmbedding').set_weights([embeddings])
    #new_model.get_layer(name='TermEmbedding').trainable = train_embeddings 
    [l for l in new_model.layers if type(l) == Model][0].get_layer(name='TermEmbedding').set_weights([embeddings])
    [l for l in new_model.layers if type(l) == Model][0].get_layer(name='TermEmbedding').trainable = train_embeddings
        
    phi_projections = [l for l in new_model.layers if l.name.startswith('Phi')]    
    for idx, phi_projection in enumerate(phi_projections):
        phi_projection.set_weights([dense[idx]])
        phi_projection.trainable = train_phi
            
    new_model.get_layer(name='Prediction').set_weights(lr_weights)
    new_model.get_layer(name='Prediction').trainable = train_sigmoid
            
    adam = Adam(lr=learning_rate, beta_1 = 0.9, beta_2 = 0.9, clipnorm=1.)
    new_model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])    
        
    return new_model
    
    

### The training algorithm incorporates mini-batch stochastic descent and negative sampling

In [19]:
def train(model,       # the model which parameters will be learnt
          epochs,      # number of epochs to run          
          batch_size,  # size of mini-batch
          m,           # number of negative samples
          data,        # data required for training                              
          neg_strategy,
          prediction_algol = crim_get_hypernym,
          bag = False
         ):

            
    # create sequences
    term_train_seq = data.tokenizer.texts_to_sequences(data.train_query)
    hyper_train_seq = data.tokenizer.texts_to_sequences(data.train_hyper)

    term_test_seq = data.tokenizer.texts_to_sequences(data.valid_query)
    hyper_test_seq = data.tokenizer.texts_to_sequences(data.valid_hyper)
                    
    validation_samples = np.arange(len(term_test_seq))
    samples = np.arange(len(term_train_seq))    
    if bag:
        # bag samples    
        samples = np.random.choice(samples, size=len(samples), replace=True)
    
    # train algorithm
    for epoch in range(epochs):
        # reset loss
        loss = 0.                
        np.random.shuffle(samples)

        shuffled_X_term, shuffled_X_hyper =\
            np.array(term_train_seq, dtype='int32')[samples],\
            np.array(hyper_train_seq, dtype='int32')[samples]

        for b in range(0, len(samples), batch_size):
            # product mini-batch, consisting of 32 +ve samples
            batch_X_term = shuffled_X_term[b:b + batch_size] 
            batch_X_hyper = shuffled_X_hyper[b:b + batch_size]

            # complement +ve samples with negatives
            batch_X_term, batch_X_hyper, batch_y_label =\
            extend_batch_with_negatives(batch_X_term, batch_X_hyper,
                                        neg_strategy,
                                        data.tokenizer, m
                                       )                                   

            # train on batch
            loss += model.train_on_batch([batch_X_term, batch_X_hyper], 
                                          batch_y_label)[0]
                    
        # model trained for full epoch       
        # pick batch of shuffled test instances with size equal to training batch
        batch_X_test_term, batch_X_test_hyper =\
                np.array(term_test_seq, dtype='int32')[validation_samples],\
                np.array(hyper_test_seq, dtype='int32')[validation_samples]
        
        batch_y_test_label = [1] * batch_X_test_term.shape[0]
        # distort test batch with some negatives to check how algorithm fares with
        # negatives            
        #batch_X_test_term, batch_X_test_hyper, batch_y_test_label =\
        #extend_batch_with_negatives(batch_X_test_term, batch_X_test_hyper,
        #                            neg_strategy,
        #                            data.tokenizer, m
        #                           )            
        
        test_loss, test_accuracy = model.test_on_batch([batch_X_test_term, batch_X_test_hyper], 
                                          batch_y_test_label)
        
        
        # compute MRR on validation set
        predictions = predict_crim_hypernyms(data.valid_query, data.tokenizer, model, prediction_algol)
        _, all_scores = get_evaluation_scores((data.valid_query, data.valid_hyper), predictions)
        mrr = round(sum([score_list[0] for score_list in all_scores]) / len(all_scores), 5)                                
                    
        print('Epoch:', epoch+1, 'Loss:', loss, 'Test Loss:', test_loss, 'MRR:', mrr, 'Test accuracy:', test_accuracy)    


In [29]:
# training harness

from tensorflow.keras.initializers import RandomNormal, Zeros, Ones
from tensorflow.keras.regularizers import l2, l1, l1_l2

#rand_norm_m0_sd001 = RandomNormal(mean = 0.0, stddev=0.01, seed=42)
#rand_norm = RandomNormal(mean = 0.0, stddev=1., seed=42)

# negative sampling options
neg_sampling_options = {'synonym':data.synonyms,                                                 
                        'random':data.random_words
                       }

# phi random init options
phi_init_options = {'random_plus_identity': RandomPlusIdentity(),
                    'random_identity': RandomIdentity(), 
                   }

kernel_constraints = {'None': None, 'ForceToOne': ForceToOne()}

# positive batch size
batch_size = 32

# implement mini-batch stochastic training
epochs = 6

# number of negative samples
m = 10
# number of projections
phi_k = 1
# train (True) or freeze
train_embeddings = False
# negative sample strategy
negative_option = 'random'
# initialise phi strategy
phi_init_option = 'random_identity'
# constrain LR parameter
kernel_constraint_option = 'None'
# dropout rate
dropout_rate = 0.3
learning_rate = 0.001
bag  = False
normalised = True
max_or_combine = True

np.random.seed(10)

embeddings_layer = get_embeddings_model(embedding_matrix=data.embedding_matrix)
# create model
crim_model = get_CRIM_model(phi_k = phi_k, train_embeddings = train_embeddings,
                            embeddings_dim = data.embeddings_dim, 
                            #vocab_size = len(data.tokenizer.word_counts), embedding = data.embedding_matrix,
                            embeddings_layer = embeddings_layer,
                            phi_init = phi_init_options[phi_init_option],                            
                            sigmoid_kernel_regularisation = None, #l2(0.001),
                            sigmoid_bias_regularisation = None, #l2(0.001),
                            sigmoid_kernel_constraint = kernel_constraints[kernel_constraint_option],
                            dropout_rate = dropout_rate,
                            learning_rate = learning_rate,
                            normalised = normalised,
                            max_or_combine = max_or_combine
                           )

crim_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Hyponym (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
Hypernym (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
model_4 (Model)                 [(None, 1, 200), (No 43904800    Hyponym[0][0]                    
                                                                 Hypernym[0][0]                   
__________________________________________________________________________________________________
Dropout_Hypo (Dropout)          (None, 1, 200)       0           model_4[1][0]                    
__________

In [ ]:
# plot model
from keras.utils.vis_utils import plot_model
#from tensorflow.keras.utils import plot_model

plot_model(crim_model, to_file='CRIM_alternate_model.png', show_shapes=True, show_layer_names=True)

In [30]:
import math

predict = True

print ('Epochs: ', epochs, 'Batch size: ', batch_size, 'm: ', m, 'pki_k: ', phi_k, 'train_embeddings: ', train_embeddings,
      'Negative sampling: ', negative_option, 'Phi Init: ', phi_init_option, 'Dropout rate: ', dropout_rate, 
      'Kernel constraint: ', kernel_constraint_option, 'Learning rate: ', learning_rate, 
      'Bagging:', bag, 'Normalised: ', normalised, 'max_or_combine: ', max_or_combine)

train(crim_model, epochs, batch_size, m, data, neg_sampling_options[negative_option], crim_max_get_hypernym)

# evaluate
if predict:
    print ("Generating predictions...")    
    crim_predictions = predict_crim_hypernyms(data.test_query, data.tokenizer, crim_model, crim_max_get_hypernym)

    print ("CRIM evaluation:")
    score_names, all_scores = get_evaluation_scores((data.test_query, data.test_hyper), crim_predictions)
    for k in range(len(score_names)):
        print (score_names[k]+': '+str(round(sum([score_list[k] for score_list in all_scores]) / len(all_scores), 5)))



('Epochs: ', 6, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.001, 'Bagging:', False, 'Normalised: ', True, 'max_or_combine: ', True)
('Epoch:', 1, 'Loss:', 220.35547143220901, 'Test Loss:', 0.9784799, 'MRR:', 0.0, 'Test accuracy:', 0.0)
('Epoch:', 2, 'Loss:', 169.29875203967094, 'Test Loss:', 1.2827737, 'MRR:', 0.0, 'Test accuracy:', 0.0)
('Epoch:', 3, 'Loss:', 135.76979410648346, 'Test Loss:', 1.579487, 'MRR:', 0.00333, 'Test accuracy:', 0.0)
('Epoch:', 4, 'Loss:', 115.8390389084816, 'Test Loss:', 1.7987796, 'MRR:', 0.02833, 'Test accuracy:', 0.0)
('Epoch:', 5, 'Loss:', 105.12239652872086, 'Test Loss:', 1.8679018, 'MRR:', 0.1, 'Test accuracy:', 0.0)
('Epoch:', 6, 'Loss:', 99.66103854775429, 'Test Loss:', 1.8479438, 'MRR:', 0.13, 'Test accuracy:', 0.0)
Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)

In [31]:
#crim_model.get_layer(name='DotProduct').get_config()
print len([k for k, v in crim_predictions.items() if k in v])

#[k for k, v in crim_predictions.items() if k in v]


crim_predictions



137


{u'scouter': [u'sports_organization',
  u'sports',
  u'athlete',
  u'sport',
  u'roster',
  u'professional_sports',
  u'basketball',
  u'team_sport',
  u'professional_football',
  u'wheelchair_basketball',
  u'sportsmanship',
  u'athletic_competition',
  u'soccer',
  u'sporting',
  u'football'],
 u'gatekeeper': [u'computer_system',
  u'computer_network',
  u'local_area_network',
  u'service_provider',
  u'personal_computer',
  u'application_software',
  u'telecommunication',
  u'software',
  u'computer',
  u'telecommunications_network',
  u'software_application',
  u'data_communication',
  u'applications_software',
  u'end-user',
  u'services'],
 u'mackerel': [u'edible',
  u'fish',
  u'shellfish',
  u'plant_part',
  u'food_product',
  u'molluscan',
  u'algae',
  u'freshwater_fish',
  u'plant_food',
  u'brine_shrimp',
  u'plant_material',
  u'oysters',
  u'fish_species',
  u'plankton',
  u'bivalve'],
 u'prefix': [u'application_software',
  u'input/output',
  u'software_application',
  u

In [53]:
hypo_seq = data.tokenizer.texts_to_sequences(data.train_query)
hyper_seq = data.tokenizer.texts_to_sequences(data.train_hyper)

train_pred_1 = crim_model.predict([hypo_seq, hyper_seq]).flatten()

In [57]:
train_pred_2 = crim_model.predict([hypo_seq, hyper_seq]).flatten()

In [66]:
print Counter(train_pred_2 > train_pred_1)


train_pred_2[100], train_pred_1[100]
len(train_pred_1)


Counter({True: 7162, False: 4617})


11779

## Start second phase of training

In [32]:
# run further training if required
m = 10
negative_option = 'random'
epochs = 2
batch_size = 32
dropout_rate = 0.3
learning_rate = 0.0005

predict = True
resume = False

if resume == False:        
    print "Create transfer learning model"
    crim_model_2 = get_CRIM_model_freeze_phi(crim_model,
                                             train_phi = False,
                                             train_embeddings = True,
                                             dropout_rate = dropout_rate,
                                             learning_rate = learning_rate)                                             

print ('Epochs: ', epochs, 'Batch size: ', batch_size, 'm: ', m, 'pki_k: ', phi_k, 'train_embeddings: ', train_embeddings,
      'Negative sampling: ', negative_option, 'Phi Init: ', phi_init_option, 'Dropout rate: ', dropout_rate, 
      'Kernel constraint: ', kernel_constraint_option, 'Learning rate: ', learning_rate, 'Bag:', bag)

train(crim_model_2, epochs, batch_size, m, data, neg_sampling_options[negative_option], crim_max_get_hypernym, bag=bag)
if predict:
    print ("Generating predictions...")
    crim_predictions_2 = predict_crim_hypernyms(data.test_query, data.tokenizer, crim_model_2, crim_max_get_hypernym)

    print ("CRIM evaluation:")
    score_names, all_scores = get_evaluation_scores((data.test_query, data.test_hyper), crim_predictions_2)
    for k in range(len(score_names)):
        print (score_names[k]+': '+str(round(sum([score_list[k] for score_list in all_scores]) / len(all_scores), 5)))



Create transfer learning model
('Epochs: ', 2, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.0005, 'Bag:', False)
('Epoch:', 1, 'Loss:', 91.90591688454151, 'Test Loss:', 1.619671, 'MRR:', 0.244, 'Test accuracy:', 0.0)
('Epoch:', 2, 'Loss:', 79.96274498105049, 'Test Loss:', 1.3946604, 'MRR:', 0.26389, 'Test accuracy:', 0.03)
Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 1400)
CRIM evaluation:
MRR: 0.30296
MAP: 0.14996
P@1: 0.26217
P@5: 0.14275
P@10: 0.13407


## Threw away affine layer weights, locked down on phi weights
('Epochs: ', 2, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 10, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.0005)

/home/jfarrugia/venv/local/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "

('Epoch:', 1, 'Loss:', 232.49733012914658, 'Test Loss:', 0.8287846446037292, 'MRR:', 0.09667)
('Epoch:', 2, 'Loss:', 199.24632835388184, 'Test Loss:', 0.9277108907699585, 'MRR:', 0.1882)

MRR: 0.27108
MAP: 0.13106
P@1: 0.23215
P@5: 0.12406
P@10: 0.117



## Third phase of training?

In [50]:
# run further training if required
m = 10
negative_option = 'random'
epochs = 5
batch_size = 1
dropout_rate = 0.3
learning_rate = 0.0001
train_embeddings = False
train_phi = True
predict = True

# transfer weights to new model
crim_model_3 = get_CRIM_model_freeze_phi(crim_model_2,
                                             train_phi = train_phi,
                                             train_embeddings = train_embeddings,
                                             dropout_rate = dropout_rate,
                                             learning_rate = learning_rate)

crim_model_3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Hyponym (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
Hypernym (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
model_6 (Model)                 [(None, 1, 200), (No 43904800    Hyponym[0][0]                    
                                                                 Hypernym[0][0]                   
__________________________________________________________________________________________________
Dropout_Hypo (Dropout)          (None, 1, 200)       0           model_6[1][0]                    
__________

In [51]:
print ('Epochs: ', epochs, 'Batch size: ', batch_size, 'm: ', m, 'pki_k: ', phi_k, 'train_embeddings: ', train_embeddings,
      'Negative sampling: ', negative_option, 'Phi Init: ', phi_init_option, 'Dropout rate: ', dropout_rate, 
      'Kernel constraint: ', kernel_constraint_option, 'Learning rate: ', learning_rate)

train(crim_model_3, epochs, batch_size, m, data, neg_sampling_options[negative_option], crim_max_get_hypernym)
if predict:
    print ("Generating predictions...")
    crim_predictions_3 = predict_crim_hypernyms(data.test_query, data.tokenizer, crim_model_3, crim_max_get_hypernym)

    print ("CRIM evaluation:")
    score_names, all_scores = get_evaluation_scores((data.test_query, data.test_hyper), crim_predictions_3)
    for k in range(len(score_names)):
        print (score_names[k]+': '+str(round(sum([score_list[k] for score_list in all_scores]) / len(all_scores), 5)))


('Epochs: ', 5, 'Batch size: ', 1, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.0001)
('Epoch:', 1, 'Loss:', 2646.805278584361, 'Test Loss:', 1.389255, 'MRR:', 0.16967, 'Test accuracy:', 0.05)
('Epoch:', 2, 'Loss:', 2418.4388537034392, 'Test Loss:', 1.2388388, 'MRR:', 0.20389, 'Test accuracy:', 0.225)
('Epoch:', 3, 'Loss:', 2214.004394464195, 'Test Loss:', 1.1100814, 'MRR:', 0.17433, 'Test accuracy:', 0.33)
('Epoch:', 4, 'Loss:', 2037.8692606203258, 'Test Loss:', 1.0190609, 'MRR:', 0.18, 'Test accuracy:', 0.4)
('Epoch:', 5, 'Loss:', 1882.0565821416676, 'Test Loss:', 0.94320244, 'MRR:', 0.18367, 'Test accuracy:', 0.475)
Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 14

In [109]:
for q, h in zip(data.train_query, data.train_hyper):
    hyp_prob = crim_model_2.predict([[data.tokenizer.word_index[q]], [data.tokenizer.word_index[h]] ])
    print q, h, hyp_prob

blackfly homopterous_insect [[0.13822134]]
blackfly insect [[0.31352118]]
turonian technical_specification [[0.23077463]]
turonian geologic_timescale [[0.17301801]]
turonian physical_property [[0.24317496]]
turonian geological_period [[0.22001244]]
turonian magnitude [[0.25076795]]
turonian unit_of_time [[0.19958243]]
turonian geological_time [[0.29483446]]
turonian geologic_time [[0.3147337]]
abhorrence distaste [[0.26146114]]
abhorrence hatred [[0.301238]]
abhorrence hate [[0.23270667]]
abhorrence disgust [[0.25828835]]
tropical_storm atmosphere [[0.30975243]]
tropical_storm windstorm [[0.3356372]]
tropical_storm violent_storm [[0.3436602]]
tropical_storm air_current [[0.27681932]]
tropical_storm atmospheric_state [[0.28246477]]
tropical_storm density [[0.16726443]]
tropical_storm current_of_air [[0.28305537]]
tropical_storm storm_damage [[0.2825659]]
tropical_storm atmospheric_phenomenon [[0.3401734]]
tropical_storm storm [[0.3732807]]
tropical_storm cyclone [[0.32870743]]
tropical_

murder_one motion_picture [[0.3930161]]
murder_one picture_show [[0.33170384]]
emmett_tyrrell newspaper_columnist [[0.2642208]]
emmett_tyrrell writer [[0.41461605]]
emmett_tyrrell person [[0.50902945]]
emmett_tyrrell columnist [[0.25528604]]
emmett_tyrrell editorialist [[0.20438492]]
emmett_tyrrell media_professional [[0.29104128]]
emmett_tyrrell journalist [[0.31737348]]
failure_rate rate [[0.20301421]]
failure_rate ratio [[0.18220463]]
kim_il-sung_university ceremony [[0.19561918]]
kim_il-sung_university school [[0.29930297]]
kim_il-sung_university academy [[0.2289273]]
kim_il-sung_university university [[0.27446017]]
kim_il-sung_university educational_institution [[0.32194445]]
kim_il-sung_university ceremonial_occasion [[0.14651176]]
chess party_game [[0.269398]]
chess parlor_game [[0.19867644]]
chess board_game [[0.31289226]]
chess parlour_game [[0.12833712]]
sachs_harbour city [[0.32362217]]
sachs_harbour hamlet [[0.15060624]]
myrdal economist [[0.18119095]]
myrdal politician [[0

spinal_fusion life_science [[0.18817279]]
spinal_fusion orthopedic_surgery [[0.32129806]]
spinal_fusion md [[0.27630508]]
spinal_fusion orthopaedics [[0.2974583]]
spinal_fusion orthopedist [[0.31243858]]
spinal_fusion doctor [[0.35547385]]
spinal_fusion medical_doctor [[0.33681884]]
spinal_fusion surgical_operation [[0.21601234]]
spinal_fusion medical_procedure [[0.31789145]]
spinal_fusion orthopaedist [[0.2543239]]
spinal_fusion orthopedics [[0.33698484]]
spinal_fusion specialist_degree [[0.20291413]]
spinal_fusion treatment [[0.28592062]]
spinal_fusion dr. [[0.28194568]]
spinal_fusion medical_care [[0.3435534]]
spinal_fusion practice_of_medicine [[0.34872663]]
spinal_fusion surgical_process [[0.20340286]]
spinal_fusion medical_specialist [[0.31877825]]
trust indicator [[0.16326107]]
trust partnership [[0.3667279]]
trust private_corporation [[0.29238474]]
trust corporation [[0.402647]]
trust company [[0.36188436]]
trust syndicate [[0.22269708]]
trust comradeship [[0.14144793]]
trust c

vendue sale [[0.30768088]]
vendue seller [[0.31579652]]
vendue salesman [[0.2374609]]
vendue selling_technique [[0.21348847]]
leaf_curl illness [[0.28790316]]
leaf_curl plant_disease [[0.30424783]]
leaf_curl sickness [[0.23860763]]
leaf_curl disease [[0.32648575]]
leaf_curl phytopathology [[0.17992261]]
leaf_curl plant_pathology [[0.25151938]]
leaf_curl phytopathogen [[0.1292862]]
leaf_curl disorder [[0.231635]]
crisis difficulty [[0.17936124]]
crisis state [[0.3365583]]
curiosa medium [[0.22006471]]
curiosa book [[0.262291]]
curiosa piece_of_work [[0.28577852]]
curiosa communication_medium [[0.3355832]]
rainfall phenomenon [[0.28784811]]
rainfall downfall [[0.09283576]]
rainfall optical_phenomenon [[0.14846677]]
rainfall natural_phenomenon [[0.36858648]]
rainfall atmospheric_precipitation [[0.21055134]]
rainfall weather [[0.36159596]]
rainfall physical_phenomenon [[0.33717924]]
rainfall weather_condition [[0.3548077]]
rainfall precipitation [[0.31402424]]
endemic_goitre disease [[0.32

course mathematical_process [[0.21270032]]
course route [[0.20126326]]
course steering [[0.16490266]]
course piloting [[0.22015883]]
escutcheon plate [[0.1538961]]
escutcheon personal_armor [[0.19113107]]
escutcheon suit_of_armour [[0.16316023]]
escutcheon armour [[0.1722294]]
escutcheon protection [[0.15596355]]
escutcheon protective_cover [[0.16576788]]
escutcheon suit_of_armor [[0.17236999]]
escutcheon protective_covering [[0.17377162]]
escutcheon body_armour [[0.19911815]]
escutcheon body_armor [[0.21284625]]
suspension supporting [[0.25012743]]
suspension mixture [[0.09946257]]
suspension technical_specification [[0.3200718]]
suspension abstract_object [[0.19791186]]
suspension support [[0.2382636]]
suspension thermodynamic_system [[0.1325794]]
suspension mechanism [[0.30134764]]
suspension state [[0.26529709]]
suspension mechanical_assembly [[0.19331318]]
suspension physical_system [[0.23167461]]
suspension inaction [[0.21673264]]
suspension pure_mathematics [[0.14825869]]
suspen

...

guts structural_biology [[0.17477098]]
guts anatomy [[0.22144197]]
guts tummy [[0.16142333]]
guts body_part [[0.2490205]]
guts breadbasket [[0.10154213]]
guts fortitude [[0.11637159]]
guts internal_organ [[0.19934407]]
guts natural_phenomenon [[0.1697211]]
narcolepsy illness [[0.37801868]]
narcolepsy sickness [[0.33250934]]
narcolepsy pathological_state [[0.2562094]]
narcolepsy disease [[0.36312813]]
narcolepsy sleep_disorder [[0.2769475]]
narcolepsy sleep_disturbance [[0.2499054]]
narcolepsy disorder [[0.3545897]]
citizen person [[0.4626088]]
citizen national [[0.26345465]]
ley grassland [[0.13283934]]
ley meadow [[0.11158025]]
hermes celestial_body [[0.21221815]]
hermes app [[0.15107669]]
hermes spacefaring [[0.22030967]]
hermes imaginary_being [[0.14218329]]
hermes greek_deity [[0.11487965]]
hermes applications_programme [[0.21856776]]
hermes spacecraft [[0.2267815]]
hermes heavenly_body [[0.16696385]]
hermes deity [[0.17915937]]
hermes godhood [[0.16374819]]
hermes space_flight [[0

hair body_part [[0.30389434]]
hair body_covering [[0.17780083]]
hair bodily_structure [[0.21338508]]
growth evolution [[0.1992704]]
ostracism proscription [[0.2484341]]
ostracism ejection [[0.10586496]]
recommendation text_file [[0.17251928]]
recommendation approval [[0.28446484]]
recommendation advice [[0.22721113]]
recommendation textfile [[0.1458871]]
empire political_organization [[0.3694939]]
empire form_of_government [[0.34003958]]
empire monarchy [[0.2833547]]
empire polity [[0.34257892]]
empire political_system [[0.35935494]]
empire regime [[0.23623496]]
empire social_control [[0.31894243]]
empire corporation [[0.21157125]]
empire political_organisation [[0.31838056]]
empire enterprise [[0.2379262]]
empire religious_ritual [[0.24057075]]
econometrics statistics [[0.25868797]]
econometrics political_economy [[0.21208026]]
econometrics statistical_analysis [[0.23468913]]
econometrics economics [[0.27605557]]
econometrics economic_science [[0.21255134]]
transport travel [[0.319359

emperor sovereign [[0.2958062]]
emperor alcoholic_beverage [[0.17150587]]
emperor person [[0.47043538]]
emperor monarch [[0.22570063]]
documentary picture [[0.26867768]]
documentary library_science [[0.26284787]]
documentary show [[0.306413]]
jeremy_nathans academician [[0.22491258]]
jeremy_nathans faculty_member [[0.27402416]]
jeremy_nathans professor [[0.26490438]]
jeremy_nathans educator [[0.33805898]]
jeremy_nathans population_genetics [[0.15056454]]
jeremy_nathans lector [[0.15074696]]
jeremy_nathans assistant_professor [[0.2419444]]
jeremy_nathans person [[0.47275808]]
jeremy_nathans educationist [[0.22943935]]
jeremy_nathans blood [[0.16726463]]
jeremy_nathans faculty [[0.24952434]]
jeremy_nathans instructor [[0.25056562]]
jeremy_nathans teacher [[0.32092863]]
stakes collective_investment_scheme [[0.21146363]]
stakes investment_funds [[0.28466403]]
stakes assets [[0.3136504]]
stakes investment [[0.3033701]]
stakes investment_fund [[0.28526]]
trust_company partnership [[0.3513042

mario_kart_ds video_game [[0.33516523]]
mario_kart_ds computing_machine [[0.2759918]]
mario_kart_ds computing_platform [[0.32606775]]
mario_kart_ds software [[0.43543056]]
official adjudicator [[0.21795204]]
official person [[0.47348452]]
official umpire [[0.17617905]]
official referee [[0.2102096]]
official football_official [[0.13984326]]
negritude penchant [[0.18314767]]
negritude social_action [[0.23926353]]
negritude proclivity [[0.19460346]]
negritude predisposition [[0.18082124]]
negritude propensity [[0.17153233]]
negritude sociology [[0.2297919]]
negritude front [[0.12553811]]
negritude social_movement [[0.23489143]]
negritude inclination [[0.16408029]]
negritude tendency [[0.1991356]]
negritude movement [[0.21804056]]
walter_boyce person [[0.42883492]]
nursery_rhyme story [[0.23292066]]
nursery_rhyme rhyming [[0.22407012]]
nursery_rhyme narrative_mode [[0.22618173]]
nursery_rhyme narrative [[0.265665]]
nursery_rhyme rhyme [[0.2220684]]
nursery_rhyme tale [[0.16947605]]
nurser

robert_de_niro thespian [[0.32112962]]
robert_de_niro film_director [[0.39441746]]
robert_de_niro filming [[0.3346187]]
robert_de_niro film_producer [[0.38015106]]
robert_de_niro actor [[0.43421277]]
robert_de_niro filmmaker [[0.40226525]]
robert_de_niro person [[0.35858992]]
robert_de_niro film_making [[0.3404514]]
robert_de_niro film_maker [[0.35280105]]
robert_de_niro movie_maker [[0.2275527]]
piastre monetary_unit [[0.2536142]]
piastre subunit [[0.08948779]]
coupling sexual_practice [[0.14042279]]
coupling bodily_process [[0.193746]]
coupling connection [[0.19608809]]
coupling connecter [[0.11518468]]
coupling connective [[0.17693052]]
coupling connector [[0.1750318]]
coupling sexual_act [[0.1686843]]
coupling sexual_activity [[0.15184824]]
coupling mechanical_assembly [[0.22553618]]
coupling mechanism [[0.29655877]]
coupling sex_activity [[0.13320418]]
coupling bodily_function [[0.19018139]]
coupling sexual_relations [[0.12525706]]
coupling connexion [[0.13038546]]
coupling human_

In [107]:
hypo_seq = data.tokenizer.texts_to_sequences(data.train_query)
hyper_seq = data.tokenizer.texts_to_sequences(data.train_hyper)

train_pred = crim_model.predict([hypo_seq, hyper_seq]).flatten()
print train_pred.shape
train_pred[train_pred < 0.5].shape

(11779,)


(11779,)

In [31]:
# save models
#crim_model.save('models/crim_phi1_notune.h5')
#crim_model_2.save('models/crim_phi1_tuned.h5')


In [33]:
crim_predictions_2

{u'scouter': [u'person',
  u'sport',
  u'leader',
  u'athlete',
  u'sportsperson',
  u'writer',
  u'competitor',
  u'olympic_sports',
  u'work_of_art',
  u'social_event',
  u'olympic_sport',
  u'actor',
  u'movie',
  u'television_show',
  u'type_of_sport'],
 u'gatekeeper': [u'person',
  u'software',
  u'computer_software',
  u'company',
  u'computer_system',
  u'computer_network',
  u'software_program',
  u'software_application',
  u'computer',
  u'applications_software',
  u'enterprise',
  u'corporation',
  u'software_package',
  u'technology',
  u'communication_medium'],
 u'mackerel': [u'fish',
  u'edible',
  u'animal',
  u'plant',
  u'plant_material',
  u'plant_part',
  u'shellfish',
  u'algae',
  u'plant_food',
  u'animal_tissue',
  u'seaweed',
  u'brine_shrimp',
  u'food_product',
  u'bivalve',
  u'crustacea'],
 u'prefix': [u'software',
  u'computer_software',
  u'software_package',
  u'computer_program',
  u'software_program',
  u'computer_code',
  u'coding_system',
  u'code',
  

In [53]:
#print np.sum(embeddings_layer.get_layer(name='TermEmbedding').get_weights()[0])

print np.sum([l for l in crim_model_2.layers if type(l) == Model][0].get_layer(name='TermEmbedding').get_weights()[0])
print np.sum(data.embedding_matrix)

[l for l in crim_model_2.layers if type(l) == Model][0]


-30727.87
-32148.61


# Ensemble model running on bagged examples

In [82]:
# train 5 models on bagged data
cluster_list = []
for i in range(5):
    print "Training", i, "model"
    cluster_list.append(get_CRIM_model(phi_k = phi_k, 
                                       train_embeddings = train_embeddings,
                                       embeddings_dim = data.embeddings_dim,                             
                                       embeddings_layer = embeddings_layer,
                                       phi_init = phi_init_options[phi_init_option],
                                       sigmoid_kernel_constraint = kernel_constraints[kernel_constraint_option],
                                       dropout_rate = dropout_rate,
                                       learning_rate = learning_rate))
    train(cluster_list[i], epochs, batch_size, m, data, neg_sampling_options[negative_option], crim_max_get_hypernym, bag=True)

Training 0 model
('Epoch:', 1, 'Loss:', 222.6102659702301, 'Test Loss:', 0.9702204, 'MRR:', 0.0, 'Test accuracy:', 0.0)
('Epoch:', 2, 'Loss:', 170.66988277435303, 'Test Loss:', 1.2822708, 'MRR:', 0.0, 'Test accuracy:', 0.0)
('Epoch:', 3, 'Loss:', 136.55550736188889, 'Test Loss:', 1.5928928, 'MRR:', 0.0, 'Test accuracy:', 0.0)
('Epoch:', 4, 'Loss:', 116.25431981682777, 'Test Loss:', 1.8125272, 'MRR:', 0.02, 'Test accuracy:', 0.0)
('Epoch:', 5, 'Loss:', 105.13798907399178, 'Test Loss:', 1.8739722, 'MRR:', 0.08667, 'Test accuracy:', 0.0)
('Epoch:', 6, 'Loss:', 99.2387764453888, 'Test Loss:', 1.862023, 'MRR:', 0.09533, 'Test accuracy:', 0.0)
('Epoch:', 7, 'Loss:', 96.11313870549202, 'Test Loss:', 1.7641302, 'MRR:', 0.1, 'Test accuracy:', 0.0)
('Epoch:', 8, 'Loss:', 93.698451384902, 'Test Loss:', 1.6952475, 'MRR:', 0.11222, 'Test accuracy:', 0.0)
('Epoch:', 9, 'Loss:', 91.52098897099495, 'Test Loss:', 1.6259766, 'MRR:', 0.11217, 'Test accuracy:', 0.0)
('Epoch:', 10, 'Loss:', 89.198438540101

In [84]:
print ("Generating predictions...")
cluster_crim_predictions = predict_cluster_hypernyms(data.test_query, data.tokenizer, cluster_list)

print ("CRIM evaluation:")
score_names, all_scores = get_evaluation_scores((data.test_query, data.test_hyper), cluster_crim_predictions)
for k in range(len(score_names)):
    print (score_names[k]+': '+str(round(sum([score_list[k] for score_list in all_scores]) / len(all_scores), 5)))


Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 1400)
CRIM evaluation:
MRR: 0.19501
MAP: 0.09175
P@1: 0.15877
P@5: 0.08695
P@10: 0.08223


In [85]:
for c in cluster_list:
    print ("Generating predictions...")
    crim_predictions = predict_crim_hypernyms(data.test_query, data.tokenizer, c, crim_max_get_hypernym)

    print ("CRIM evaluation:")
    score_names, all_scores = get_evaluation_scores((data.test_query, data.test_hyper), crim_predictions)
    for k in range(len(score_names)):
        print (score_names[k]+': '+str(round(sum([score_list[k] for score_list in all_scores]) / len(all_scores), 5)))

Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 1400)
CRIM evaluation:
MRR: 0.17378
MAP: 0.0815
P@1: 0.13742
P@5: 0.07843
P@10: 0.07263
Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 1400)
CRIM evaluation:
MRR: 0.16705
MAP: 0.0779
P@1: 0.13276
P@5: 0.07427
P@10: 0.07028
Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 1400)
CRIM evaluation:
MRR: 0.18839
MAP: 0.08784
P@1: 0.15077
P@5: 0.08357
P@10: 0.07837
Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Do

## Evaluation  code

In [ ]:
# test whether two words are related by hypernymy
i = data.tokenizer.word_index['ebert']
j = data.tokenizer.word_index['writer']
crim_model_2.predict([[i], [j]])


### Find candidate hypernyms

In [69]:
class ToyClass:
    def __init__(self, **kwargs):
        print kwargs['p1']
        if 'p2' in kwargs:
            print kwargs['p2']
            
ToyClass(p1='hello',p2='goodbye' )


hello
goodbye


<__main__.ToyClass instance at 0x7fc745037c68>

# Try to refine initial training with clustering

In [131]:
class CrimCluster:
    def __init__(self, **kwargs):                              
        
        # create Keras model
        if 'model' in kwargs:
            # we're passing a ready-trained model here
            self.model = kwargs['model']
        else:            
            self.model = self._init_model(phi_init = kwargs['phi_init'], 
                                          embeddings_layer = kwargs['embeddings_layer'],
                                          embeddings_dim = kwargs['embeddings_dim'],
                                          sigmoid_kernel_constraint = kwargs['sigmoid_kernel_constraint'],
                                          dropout_rate  = kwargs['dropout_rate'],
                                          learning_rate = kwargs['learning_rate'])              
        # initialise variables     
        self.epoch_count = 0
        self.epoch_total_loss = 0.
        self.avg_loss = []        
        self.training_samples = {}
        self.mrr = []
                        
    def _init_model(self, phi_init, sigmoid_kernel_constraint, 
                    embeddings_layer, embeddings_dim,
                    dropout_rate, learning_rate):
                                                   
        return get_CRIM_model(phi_k = 1, 
                              train_embeddings = False,
                              embeddings_dim = embeddings_dim,
                              embeddings_layer = embeddings_layer,
                              phi_init = phi_init,
                              sigmoid_kernel_constraint = sigmoid_kernel_constraint,
                              dropout_rate = dropout_rate,
                              learning_rate = learning_rate,
                              normalised = False
                             )
        
    
    def calc_avg_loss(self):        
        self.avg_loss.append(self.epoch_total_loss / self.training_samples[self.epoch_count])
        
    def increment_epoch(self):
        self.epoch_count += 1
        
    def increment_training_samples(self):
        if self.epoch_count in self.training_samples:
            self.training_samples[self.epoch_count] += 1
        else:
            self.training_samples[self.epoch_count] = 1

In [144]:
# rolled-back some changes:
# removed cluster max;
# removed conditons to control creation of new cluster.  Cluster can be appended immediately


def yamane_train(
    epochs,      # number of epochs to run
    m,           # number of negative samples
    data,        # class instance containing all the data required for training/testing        
    embedding_layer,     # shared embeddings layer
    threshold    = 0.15,     # threshold; similarity below this score will trigger new cluster
    negative_option = 'random', # pass dictionary of random terms 
    phi_init_option = None,     # phi dense layer initialisation strategy
    sigmoid_constraint_option = 'None',
    dropout_rate = 0.,
    learning_rate = 0.0001,
    cluster_max = 30,
    init_model = None
): 
    
    phi_init_options = {'random_identity': RandomIdentity(),                         
                        'random_plus_identity': RandomPlusIdentity}
    
    neg_sampling_options = {'synonym':data.synonyms, 'random':data.random_words}
    
    sigmoid_constraint_options = {'ForceToOne': ForceToOne(), 'None': None}
    
    sigmoid_kernel_constraint = sigmoid_constraint_options[sigmoid_constraint_option]
    
    neg_strategy = neg_sampling_options[negative_option]
            
    # create sequences
    # we have two sets of inputs: one for training query and hypernym terms;
    #                             another for the validation query/hyper terms;
    term_train_seq = data.tokenizer.texts_to_sequences(data.train_query)
    hyper_train_seq = data.tokenizer.texts_to_sequences(data.train_hyper)

    #term_test_seq = data.tokenizer.texts_to_sequences(data.valid_query)
    #hyper_test_seq = data.tokenizer.texts_to_sequences(data.valid_hyper)        
    
    term_train_seq, hyper_train_seq = [np.asarray(x, dtype='int32') for x in [term_train_seq, hyper_train_seq]]
            
    # this list stores which cluster each training sequence pertains to
    sample_clusters = np.zeros(len(term_train_seq), dtype='int32')
    
    print ("m: ", m, "lambda: ", threshold, "max epoch per cluster: ", epochs, 
           "Negative sampling: ", negative_option, "Phi Init: ", phi_init_option,
           "sigmoid_kernel_constraint: ", sigmoid_constraint_option, 
           "dropout: ", dropout_rate, "learning_rate: ", learning_rate, 
           "cluster_max: ", cluster_max          
          )
    
    sample_count = len(sample_clusters)
    print ("Sample clusters size: ", sample_count)
    # list containing 1 model per cluster
    clusters = []    
        
    #clusters.append(CrimCluster(model=init_model))
    clusters.append(CrimCluster(phi_init = phi_init_options[phi_init_option],
                                embeddings_layer = embedding_layer,
                                embeddings_dim = data.embeddings_dim,
                                sigmoid_kernel_constraint = sigmoid_kernel_constraint,
                                dropout_rate = dropout_rate,
                                learning_rate = learning_rate))
                    
    # get training set indices
    indices = np.arange(len(term_train_seq))  
    
    # get test set indices
    #test_indices = np.arange(len(term_test_seq))
            
    # initialise each training sample to cluster 0
    sample_clusters[indices] = 0        
    
    # seed random generator
    np.random.seed(42)
    
    # indicator of "current" sample cluster index
    z_i = 0
                    
    #while np.min([c.epoch_count for c in clusters]) < epochs:
    for epoch in range(epochs):
        sample_count = len(sample_clusters)
        # reset loss for each cluster                        
        for c in clusters:
            #if c.epoch_count < epochs:                
            #    c.loss = 0.            
            c.epoch_total_loss = 0.
        
        
        # shuffle indices every epoch
        np.random.shuffle(indices)
        
        # train algorithm by stochastic gradient descent, one sample at a time
        # learn 1 matrix of first epoch only
        for idx, i in enumerate(indices):                        
            if (idx + 1) % 1000 == 0:
                print ("Processed ", idx+1, "samples...")
            
            # calculate similarity on all clusters
            sim = list(map(lambda x: x.model.predict([term_train_seq[i], hyper_train_seq[i]]), clusters))            
            max_sim = np.argmax(sim)            
                        
            #print "Term:", tokenizer.index_word[term_train_seq[i][0]], 'Hyper:', tokenizer.index_word[hyper_train_seq[i][0]], "Max Similarity cluster:", max_sim, "(sim = %0.8f)" % (sim[max_sim])
            # limit cluster creation to a given max
                       
            #if (clusters[0].epoch_count > 0 and  sim[max_sim] < threshold and len(clusters) < cluster_max): 
            if (sim[max_sim] < threshold):
                if len(clusters) >= cluster_max:
                    # cluster ceiling exceeded so skip training example                    
                    sample_clusters[i] = -1
                    continue                                            
                
                # if cluster registering best simalarity hasn't registered minimum quota of training 
                # samples, don't create new cluster
                #if clusters[max_sim].training_samples[clusters[max_sim].epoch_count] < 50:
                #    z_i = max_sim
                #    sample_clusters[i] = z_i
                #else:                
                
                # add new cluster to list of clusters                                
                clusters.append(CrimCluster(phi_init = phi_init_options[phi_init_option],
                                              embeddings_layer = embedding_layer,
                                              embeddings_dim = data.embeddings_dim,
                                              sigmoid_kernel_constraint = sigmoid_kernel_constraint,
                                              dropout_rate = dropout_rate,
                                              learning_rate = learning_rate))

                # assign current cluster index to latest model
                z_i = len(clusters) - 1
                sample_clusters[i] = z_i
            else:            
                z_i = max_sim
                sample_clusters[i] = z_i                
                        
            # allocate sample to cluster for statistics
            
            # if current cluster reached/exceeded epoch count, skip current sample (i.e don't update cluster)                        
            #if (clusters[z_i].epoch_count < epochs):                                
            
            # extend samples in cluster with negative samples
            batch_X_term, batch_X_hyper, batch_y_label =\
                extend_batch_with_negatives(term_train_seq[i], 
                                            hyper_train_seq[i],
                                            neg_strategy,
                                            data.tokenizer, m
                                           )  

            # update parameters of cluster 
            clusters[z_i].epoch_total_loss += clusters[z_i].model.train_on_batch(
                [batch_X_term, batch_X_hyper], batch_y_label)[0]            
            clusters[z_i].increment_training_samples()
        
            ####################### END OF EPOCH #######################                
        
        # instead of test loss, measure MRR as a more indicative validation metric
        print ("Running evaluation on trial data set...")
        predictions = predict_cluster_hypernyms(data.valid_query, data.tokenizer, clusters)
        _, all_scores = get_evaluation_scores((data.valid_query, data.valid_hyper), predictions)
        mrr = round(sum([score_list[0] for score_list in all_scores]) / len(all_scores), 5)
        clusters[0].mrr.append(mrr)
        
        # calculate mean loss and increase epoch_count for clusters
        
        for cluster in clusters:    
            cluster.calc_avg_loss()
            cluster.epoch_count += 1
                
        print('Epoch:', max([c.epoch_count for c in clusters]), 'Cluster #:', len(clusters) ,
              'Loss:', np.mean([c.avg_loss[::-1][0] for c in clusters]),
              'Test MRR:', mrr)
    return clusters, sample_clusters

In [146]:
import datetime

# initialise embedding later which will be shared among all clusters
#embeddings_matrix = [l for l in crim_model_2.layers if type(l) == Model][0].get_layer(name='TermEmbedding').get_weights()[0]
embeddings_matrix = data.embedding_matrix
embedding_layer = get_embeddings_model(embedding_matrix=embeddings_matrix)
epochs = 20
m = 5

print ("Training started at: %s" %  (datetime.datetime.now()))
clusters, sample_clusters =\
    yamane_train(epochs, m, 
                 data,
                 embedding_layer,
                 threshold = 0.2,
                 negative_option = 'random',
                 phi_init_option = 'random_identity',
                 sigmoid_constraint_option = 'ForceToOne',
                 dropout_rate = 0.1,
                 learning_rate = 0.001,
                 cluster_max = 40
                 #init_model = crim_model_2
                )

print ("Training concluded at: %s" % (datetime.datetime.now()))

Training started at: 2019-01-04 02:17:48.946289
('m: ', 5, 'lambda: ', 0.2, 'max epoch per cluster: ', 20, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'sigmoid_kernel_constraint: ', 'ForceToOne', 'dropout: ', 0.1, 'learning_rate: ', 0.001, 'cluster_max: ', 40)
('Sample clusters size: ', 11779)
('Processed ', 1000, 'samples...')
('Processed ', 2000, 'samples...')
('Processed ', 3000, 'samples...')
('Processed ', 4000, 'samples...')
('Processed ', 5000, 'samples...')
('Processed ', 6000, 'samples...')
('Processed ', 7000, 'samples...')
('Processed ', 8000, 'samples...')
('Processed ', 9000, 'samples...')
('Processed ', 10000, 'samples...')
('Processed ', 11000, 'samples...')
Running evaluation on trial data set...


MemoryError: 

In [147]:
from collections import Counter
#sum(Counter(sample_clusters).values()[:])
Counter(sample_clusters)

Counter({0: 9376, 1: 2290, 2: 113})

In [148]:
for c in clusters:
    print c.training_samples, c.avg_loss

{0: 9918, 1: 9377, 2: 9378, 3: 9385, 4: 9376} [0.3383950079052894, 0.1707735969544977, 0.12343727941536435, 0.0986675734687661, 0.08062875606970399]
{0: 1861, 1: 2402, 2: 2401, 3: 2368, 4: 2290} [0.596995979147147, 0.44526770622853334, 0.3491070275394706, 0.28401292040094583, 0.23139021659359402]
{0: 26, 1: 113} [0.6919059547094198, 0.6692267740722251]


In [135]:
clusters[0].model.get_layer(name='Prediction').get_weights()

[array([[1.]], dtype=float32), array([0.68318665], dtype=float32)]

In [ ]:
print ("Generating predictions...")

cluster_predictions = predict_cluster_hypernyms(data.test_query, data.tokenizer, clusters)
#cluster_predictions = predict_crim_hypernyms(data.test_query, data.tokenizer, clusters[0].model, crim_get_hypernym)

print ("CRIM evaluation:")
score_names, all_scores = get_evaluation_scores((data.test_query, data.test_hyper), cluster_predictions)
for k in range(len(score_names)):
    print (score_names[k]+': '+str(round(sum([score_list[k] for score_list in all_scores]) / len(all_scores), 5)))

In [142]:
cluster_predictions['pumpkin']

[u'amusement_ride',
 u'business_establishment',
 u'liquor_licence',
 u'commercial_enterprise',
 u'europages',
 u'toy_industry',
 u'manufacture',
 u'definition',
 u'distributor',
 u'person',
 u'purpose',
 u'main_function',
 u'licensable',
 u'society',
 u'interprovincial']

In [44]:
# Pretend we're evaluating only on one of the clusters
cluster0_predictions = predict_crim_hypernyms(data.test_query, data.tokenizer, clusters[0].model, crim_max_get_hypernym)

print ("CRIM evaluation:")
score_names, all_scores = get_evaluation_scores((data.test_query, data.test_hyper), cluster0_predictions)
for k in range(len(score_names)):
    print (score_names[k]+': '+str(round(sum([score_list[k] for score_list in all_scores]) / len(all_scores), 5)))

('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 1400)
CRIM evaluation:
MRR: 0.09468
MAP: 0.04557
P@1: 0.06338
P@5: 0.04542
P@10: 0.04246


In [99]:
print Counter(sample_clusters)

for c in clusters:
    #print c.model.get_layer(name ='Prediction').get_weights()
    print c.epoch_count, c.training_samples

#clusters[9].model.get_layer(name='Prediction').get_weights()
#predict_crim_hypernyms(['rod_laver'], data.tokenizer, crim_model, crim_max_get_hypernym)

#crim_model_2.predict([[data.tokenizer.word_index['rod_laver']], 
#                    [data.tokenizer.word_index['athlete']]])


print np.sum([l for l in clusters[0].model.layers if type(l) == Model][0].get_layer(name='TermEmbedding').get_weights()[0])
print np.sum(data.embedding_matrix)

np.sum(embedding_layer.get_layer(name='TermEmbedding').get_weights()[0])

Counter({-1: 10930, 0: 849})
5 {0: 11779, 1: 7330, 2: 2422, 3: 1208, 4: 849}
-32148.61
-32148.61


-32148.61

In [100]:
from collections import Counter
print Counter(sample_clusters)


#sample_clusters
#print np.where(sample_clusters == 1)[0]

#data.train_query[66], data.train_hyper[66]
for q, h in zip(data.train_query, data.train_hyper):
    hyp_prob = clusters[0].model.predict([[data.tokenizer.word_index[q]], [data.tokenizer.word_index[h]] ])
    print q, h, hyp_prob

Counter({-1: 10930, 0: 849})
blackfly homopterous_insect [[0.0920177]]
blackfly insect [[0.15019463]]
turonian technical_specification [[0.09434162]]
turonian geologic_timescale [[0.09426979]]
turonian physical_property [[0.10644199]]
turonian geological_period [[0.10768096]]
turonian magnitude [[0.10547616]]
turonian unit_of_time [[0.09206219]]
turonian geological_time [[0.12664953]]
turonian geologic_time [[0.13282974]]
abhorrence distaste [[0.10247174]]
abhorrence hatred [[0.11265936]]
abhorrence hate [[0.11295412]]
abhorrence disgust [[0.10763728]]
tropical_storm atmosphere [[0.11236698]]
tropical_storm windstorm [[0.1406302]]
tropical_storm violent_storm [[0.13833816]]
tropical_storm air_current [[0.10461441]]
tropical_storm atmospheric_state [[0.11180094]]
tropical_storm density [[0.08922068]]
tropical_storm current_of_air [[0.10645618]]
tropical_storm storm_damage [[0.14270832]]
tropical_storm atmospheric_phenomenon [[0.11525891]]
tropical_storm storm [[0.16198856]]
tropical_sto

deck platform [[0.10865111]]
deck shape [[0.08933691]]
deck surface [[0.09336391]]
deck venture [[0.09007652]]
deck floor [[0.09791429]]
deck upper_deck [[0.10380888]]
jet_stream phenomenon [[0.09883424]]
jet_stream air_current [[0.10835018]]
jet_stream weather [[0.15238938]]
jet_stream atmospheric_phenomenon [[0.11710775]]
jet_stream current_of_air [[0.10401927]]
jet_stream physical_phenomenon [[0.09822613]]
jet_stream weather_condition [[0.13978507]]
jet_stream airstream [[0.11032039]]
jet_stream wind [[0.13711978]]
muhammad_ali_jinnah person [[0.18955883]]
muhammad_ali_jinnah national_leader [[0.10378031]]
muhammad_ali_jinnah statesman [[0.10555199]]
muhammad_ali_jinnah politician [[0.11716806]]
muhammad_ali_jinnah owner [[0.12218209]]
muhammad_ali_jinnah political_leader [[0.11928935]]
muhammad_ali_jinnah leader [[0.10662608]]
habit function [[0.10377374]]
habit affair [[0.08908623]]
habit suit_of_clothes [[0.09431229]]
habit exercise [[0.12535225]]
habit suit [[0.10368294]]
bennin

certified_check cheque [[0.11816584]]
certified_check check [[0.10436872]]
certified_check order_of_payment [[0.11613838]]
pearl stone [[0.08660061]]
pearl ball [[0.081631]]
pearl substantial_form [[0.08347006]]
pearl abstract_object [[0.09152839]]
pearl precious_stone [[0.08760844]]
pearl jewel [[0.08992292]]
pearl antique_white [[0.07260676]]
pearl rock [[0.08500468]]
pearl pebble [[0.07896512]]
pearl orb [[0.08921589]]
pearl white [[0.08915613]]
playwriting writer [[0.12098771]]
sajid_mahmood person [[0.1928514]]
sajid_mahmood sportswoman [[0.10695394]]
sajid_mahmood competitor [[0.10709269]]
sajid_mahmood cricketer [[0.10207116]]
sajid_mahmood sportsperson [[0.12088095]]
smew sea_duck [[0.09961044]]
smew waterfowl [[0.12519911]]
smew waterbird [[0.10835594]]
smew animal [[0.17484443]]
smew duck [[0.10444371]]
biosatellite satellite [[0.0984017]]
biosatellite computer_code [[0.10752177]]
biosatellite transportation [[0.1060435]]
biosatellite electrical_appliance [[0.09764454]]
biosa

photojournalism research [[0.11636108]]
photojournalism investigation [[0.11053377]]
photojournalism piece_of_work [[0.10384881]]
photojournalism mass_media [[0.15154734]]
photojournalism narrative [[0.1136984]]
photojournalism investigating [[0.10886014]]
photojournalism print_media [[0.15385951]]
photojournalism news_media [[0.14177766]]
photojournalism communication_medium [[0.1395691]]
photojournalism book [[0.10881636]]
photojournalism mass_medium [[0.13135728]]
spaceport installation [[0.11669578]]
spaceport facility [[0.12684835]]
el_nino dramatics [[0.09181613]]
el_nino classical [[0.08272935]]
el_nino dramatic_play [[0.0946926]]
till equipment [[0.10136623]]
till parcel [[0.10107803]]
horizontal_bar durable_good [[0.09782653]]
horizontal_bar durable_goods [[0.08973133]]
horizontal_bar gymnastic_apparatus [[0.0890138]]
horizontal_bar equipment [[0.11624561]]
horizontal_bar exerciser [[0.10171664]]
horizontal_bar sports_equipment [[0.10870934]]
parameter computer_code [[0.159922

ghrelin hormone [[0.12013252]]
ghrelin humor [[0.08344194]]
ghrelin biomolecule [[0.11739194]]
ghrelin biopolymer [[0.1030577]]
ghrelin chemical_group [[0.09615467]]
ghrelin chemical_series [[0.08851977]]
ghrelin signaling_molecule [[0.11140775]]
ghrelin chemical_bond [[0.09791756]]
ghrelin internal_secretion [[0.09590401]]
ghrelin gi_hormones [[0.09301982]]
ghrelin body_fluid [[0.12444063]]
ghrelin gastrointestinal_hormone [[0.09798939]]
ghrelin moiety [[0.09555206]]
ghrelin bond [[0.09365266]]
benefaction donation [[0.11414466]]
benefaction kindness [[0.10304992]]
benefaction contribution [[0.10920805]]
benefaction gift [[0.11521389]]
abdication resignation [[0.11033426]]
abdication group_event [[0.10427248]]
abdication written_document [[0.13660353]]
tebuconazole fungicide [[0.12353768]]
heritage transferred_property [[0.11727265]]
delay code [[0.12401038]]
delay time_interval [[0.10680801]]
delay interval [[0.09456098]]
delay break [[0.08761637]]
delay computer_code [[0.11435198]]


...

hermes spacefaring [[0.09418854]]
hermes imaginary_being [[0.10666751]]
hermes greek_deity [[0.0837125]]
hermes applications_programme [[0.09681037]]
hermes spacecraft [[0.09590679]]
hermes heavenly_body [[0.08691106]]
hermes deity [[0.09485193]]
hermes godhood [[0.09698103]]
hermes space_flight [[0.09263504]]
hermes turbine [[0.09172239]]
hermes cartoon_character [[0.10245124]]
hermes aircraft_engine [[0.09616449]]
hermes orbiter [[0.09495706]]
hermes astronomical_object [[0.09184566]]
hermes application_software [[0.10612185]]
hermes video_game_console [[0.09091603]]
hermes space_travel [[0.10032665]]
hermes computing_platform [[0.10249997]]
hermes air_transportation [[0.10561773]]
hermes games_console [[0.09572623]]
hermes air_travel [[0.11049117]]
hermes manufacture [[0.10334908]]
hermes satellite [[0.09048877]]
hermes electronic_game [[0.09446153]]
hermes artificial_satellite [[0.08525581]]
hermes fictional_character [[0.10396955]]
hermes aircraft [[0.09652032]]
hermes language [[

truffle plant [[0.13750483]]
truffle plant_structure [[0.11653318]]
truffle green_goods [[0.10045905]]
truffle goody [[0.09232794]]
truffle tracheophyte [[0.0924999]]
truffle delicacy [[0.09316944]]
truffle candy [[0.10560989]]
truffle sweets [[0.1001826]]
truffle plant_organ [[0.0865583]]
truffle treat [[0.11408426]]
truffle dainty [[0.08805463]]
truffle plant_part [[0.13074304]]
truffle confection [[0.09830377]]
truffle sweet [[0.09791212]]
don_baldwin competitor [[0.09957205]]
don_baldwin athlete [[0.11448663]]
don_baldwin person [[0.18453124]]
don_baldwin skater [[0.08980215]]
don_baldwin sport [[0.09521502]]
don_baldwin sportsperson [[0.10538402]]
great-grandfather forbear [[0.09141254]]
great-grandfather antecedent [[0.0974097]]
great-grandfather ancestor [[0.09809014]]
great-grandfather paternity [[0.10362198]]
great-grandfather parent [[0.12245066]]
great-grandfather forebear [[0.09409192]]
great-grandfather relative [[0.09792958]]
great-grandfather person [[0.18954957]]
great-

angular_momentum natural_action [[0.07577547]]
angular_momentum momentum [[0.08982228]]
angular_momentum natural_process [[0.0953118]]
greccio comune [[0.0935168]]
greccio city [[0.11087576]]
shortsightedness softness [[0.08127858]]
shortsightedness visual_impairment [[0.11596639]]
shortsightedness sickness [[0.11776762]]
shortsightedness impairment [[0.1200942]]
macroeconomics economics [[0.11263908]]
ridge convex_shape [[0.07493343]]
ridge elevation [[0.10225245]]
ridge natural_elevation [[0.08492781]]
ridge structural_member [[0.08886942]]
ridge support [[0.0792508]]
ridge beam [[0.08626557]]
ridge shape [[0.07777116]]
ridge structural_element [[0.079677]]
ridge appendage [[0.07959195]]
standerton town [[0.10592004]]
standerton city [[0.11831442]]
snowplow travel [[0.12163011]]
snowplow move [[0.09083003]]
snowplow transport [[0.12399141]]
snowplow road_vehicle [[0.13514033]]
snowplow vehicle [[0.1343127]]
grapevine album [[0.09132314]]
grapevine vinifera [[0.10166208]]
grapevine pl

masquerade_ball show [[0.09789938]]
masquerade_ball masquerade [[0.09638412]]
masquerade_ball masque [[0.09531285]]
masquerade_ball mask [[0.09467316]]
masquerade_ball social_occasion [[0.11746582]]
masquerade_ball social_event [[0.12385956]]
masquerade_ball party [[0.11854265]]
masquerade_ball function [[0.09493471]]
masquerade_ball occasion [[0.12039457]]
masquerade_ball group_event [[0.11171931]]
masquerade_ball masquerade_party [[0.09744855]]
masquerade_ball social_gathering [[0.11705574]]
mechanics performance [[0.10647947]]
mechanics physics [[0.1010772]]
bearing mechanical_assembly [[0.08699782]]
bearing direction [[0.09221358]]
bearing component [[0.10748389]]
bearing support [[0.09948286]]
bearing mechanism [[0.10359461]]
carborundum abrasive [[0.09172519]]
carborundum abrasive_material [[0.08950742]]
épée sword [[0.08091982]]
épée weapon [[0.10465256]]
rubble scrap [[0.09006798]]
crusade disagreement [[0.1208376]]
crusade task [[0.11283879]]
crusade military_campaign [[0.1042

praetor magistrate [[0.12316488]]
praetor judge [[0.13197124]]
praetor person [[0.193164]]
endocarditis carditis [[0.11675758]]
endocarditis inflammation [[0.1339646]]
endocarditis disorder [[0.1260834]]
endocarditis disease [[0.15686767]]
endocarditis sickness [[0.12046529]]
michael_owen footballer [[0.10681432]]
michael_owen person [[0.19513299]]


# Some readings

* ('Epochs: ', 30, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 24, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout 0.5: ', True)

CRIM evaluation:<br>
MRR: 0.13002<br>
P@1: 0.08806<br>
P@5: 0.06368<br>
P@10: 0.06131<br>

* ('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.5, 'Kernel constraint: ', 'None')

CRIM evaluation:<br>
MRR: 0.16015<br>
P@1: 0.12675<br>
P@5: 0.07046<br>
P@10: 0.06632<br>

* ('Epochs: ', 15, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.5: ')

CRIM evaluation:<br>
MRR: 0.15522<br>
P@1: 0.11741<br>
P@5: 0.07256<br>
P@10: 0.06844<br>

* ('Epochs: ', 5, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', True, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout (0.5): ', True)

CRIM evaluation:<br>
MRR: 0.22385<br>
P@1: 0.1968<br>
P@5: 0.08815<br>
P@10: 0.08352<br>

Even though these results are superior on the outset, in reality, tuning the embeddings reduces the model to Most Frequent Hypernym.


* ('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 5, 'pki_k: ', 24, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout: ', True)

Reducing the number of random samples has a negative impact.

    CRIM evaluation:<br>
MRR: 0.06439<br>
P@1: 0.04536<br>
P@5: 0.02777<br>
P@10: 0.02703<br>

* ('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 24, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.5, 'Kernel constraint: ', 'ForceToOne')

CRIM evaluation:<br>
MRR: 0.12434<br>
P@1: 0.09006<br>
P@5: 0.05667<br>
P@10: 0.0549<br>

* ('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 5, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.5, 'Kernel constraint: ', 'None')

CRIM evaluation:<br>
MRR: 0.1346<br>
P@1: 0.0974<br>
P@5: 0.0627<br>
P@10: 0.05998<br>

* ('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 10, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.5, 'Kernel constraint: ', 'None')

CRIM evaluation:<br>
MRR: 0.12567<br>
P@1: 0.08539<br>
P@5: 0.06105<br>
P@10: 0.05916<br>


* ('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 5, 'pki_k: ', 12, 'train_embeddings: ', False, 'Negative sampling: ', 'synonym', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.5, 'Kernel constraint: ', 'None')
('Epoch:', 1, 'Loss:', 155.5154161900282, 'Test Loss:', 485.83162450790405)
('Epoch:', 2, 'Loss:', 88.82768769562244, 'Test Loss:', 348.13482135534286)
('Epoch:', 3, 'Loss:', 66.63960940390825, 'Test Loss:', 258.7310974597931)
('Epoch:', 4, 'Loss:', 56.400326274335384, 'Test Loss:', 216.1373891234398)
('Epoch:', 5, 'Loss:', 51.70219925045967, 'Test Loss:', 207.98559176921844)
('Epoch:', 6, 'Loss:', 48.9724283516407, 'Test Loss:', 204.41960680484772)
('Epoch:', 7, 'Loss:', 46.962790466845036, 'Test Loss:', 195.1675413697958)
('Epoch:', 8, 'Loss:', 45.41264865081757, 'Test Loss:', 198.74169850349426)
('Epoch:', 9, 'Loss:', 45.02060864120722, 'Test Loss:', 191.37398713827133)
('Epoch:', 10, 'Loss:', 44.76244197413325, 'Test Loss:', 188.13010711967945)

CRIM evaluation:<br>
MRR: 0.01452<br>
P@1: 0.00801<br>
P@5: 0.00653<br>
P@10: 0.00623<br>

* ('Epochs: ', 15, 'Batch size: ', 32, 'm: ', 15, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.5, 'Kernel constraint: ', 'None')
('Epoch:', 1, 'Loss:', 173.66530799865723, 'Test Loss:', 422.2526289820671)
('Epoch:', 2, 'Loss:', 89.43182794749737, 'Test Loss:', 888.1642265319824)
('Epoch:', 3, 'Loss:', 82.11764796078205, 'Test Loss:', 897.4773137569427)
('Epoch:', 4, 'Loss:', 77.61969149112701, 'Test Loss:', 871.3285944461823)
('Epoch:', 5, 'Loss:', 73.61261868476868, 'Test Loss:', 838.6195100545883)
('Epoch:', 6, 'Loss:', 70.76602215319872, 'Test Loss:', 790.632804274559)
('Epoch:', 7, 'Loss:', 68.09195621311665, 'Test Loss:', 758.9123626947403)
('Epoch:', 8, 'Loss:', 66.28909918665886, 'Test Loss:', 722.3105019330978)
('Epoch:', 9, 'Loss:', 64.0174068659544, 'Test Loss:', 701.5663638114929)
('Epoch:', 10, 'Loss:', 62.1955054551363, 'Test Loss:', 685.4047366380692)
('Epoch:', 11, 'Loss:', 61.18628938496113, 'Test Loss:', 666.7885119915009)
('Epoch:', 12, 'Loss:', 60.10102154314518, 'Test Loss:', 654.0900322198868)
('Epoch:', 13, 'Loss:', 59.095360577106476, 'Test Loss:', 653.2110993862152)
('Epoch:', 14, 'Loss:', 58.58073855936527, 'Test Loss:', 634.9157860279083)
('Epoch:', 15, 'Loss:', 57.885950952768326, 'Test Loss:', 631.2514699697495)

CRIM evaluation:
MRR: 0.15946
P@1: 0.11674
P@5: 0.07472
P@10: 0.06976

-----------------------------------------------
Tried new technique involving mix of ensemble and transfer learning.<br>
* Developed two models: one is the standard CRIM model I've always used.  The embeddings are frozen and populated with the word2vec vectors provided by Gabriel.
* After training this model for an adequate number of epochs (i.e. until optimal fitting; early stopping implemented manually);
* After the first cycle of training is over I extracted the phi and LR weights;
* These weights were subsequently injected in a new model.  This time the embeddings later was set to trainable and the Phi dense layer was frozen;
* The model was encouraged to use the same projection weights and fine-tune the embeddings to learn a better hypernym generation model;

#### 1 Projection; First Cycle 
('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None')
('Epoch:', 1, 'Loss:', 177.9488000869751, 'Test Loss:', 178.53641974925995)
('Epoch:', 2, 'Loss:', 103.15769128501415, 'Test Loss:', 102.5895478874445)
('Epoch:', 3, 'Loss:', 90.33858793973923, 'Test Loss:', 90.0393942296505)
('Epoch:', 4, 'Loss:', 80.81938941776752, 'Test Loss:', 79.73569859564304)
('Epoch:', 5, 'Loss:', 74.30441601574421, 'Test Loss:', 73.00559163093567)
('Epoch:', 6, 'Loss:', 68.31351159512997, 'Test Loss:', 67.29328979551792)
('Epoch:', 7, 'Loss:', 64.69703111797571, 'Test Loss:', 64.18054696917534)
('Epoch:', 8, 'Loss:', 61.36914176493883, 'Test Loss:', 60.54891411960125)
('Epoch:', 9, 'Loss:', 57.571076557040215, 'Test Loss:', 58.882518880069256)
('Epoch:', 10, 'Loss:', 54.78656556457281, 'Test Loss:', 57.673407919704914)
Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 1400)
CRIM evaluation:
MRR: 0.14914
P@1: 0.10874
P@5: 0.0687
P@10: 0.06505


#### 1 Projection; Second Cycle
('Epochs: ', 5, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', True, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None')
('Epoch:', 1, 'Loss:', 29.900435734540224, 'Test Loss:', 50.436300061643124)
('Epoch:', 2, 'Loss:', 8.763934534537839, 'Test Loss:', 46.93847519904375)
('Epoch:', 3, 'Loss:', 4.247505006627762, 'Test Loss:', 51.40926795452833)
('Epoch:', 4, 'Loss:', 2.347970368108463, 'Test Loss:', 56.144402757287025)
('Epoch:', 5, 'Loss:', 1.38899088197104, 'Test Loss:', 63.76580411195755)
Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 1400)
CRIM evaluation:
MRR: 0.28634
P@1: 0.24483
P@5: 0.1263
P@10: 0.11961

---------------------------------------------------------------------------------------
#### 24 Projections; First Cycle
('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 24, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None')
('Epoch:', 1, 'Loss:', 109.97628237307072, 'Test Loss:', 110.91941311955452)
('Epoch:', 2, 'Loss:', 61.31632810086012, 'Test Loss:', 65.23697778582573)
('Epoch:', 3, 'Loss:', 48.39542027562857, 'Test Loss:', 59.35354737192392)
('Epoch:', 4, 'Loss:', 41.11544480547309, 'Test Loss:', 60.305753372609615)
('Epoch:', 5, 'Loss:', 36.67992676049471, 'Test Loss:', 64.0475360751152)
('Epoch:', 6, 'Loss:', 34.28844119235873, 'Test Loss:', 63.37563705444336)
('Epoch:', 7, 'Loss:', 32.068154136650264, 'Test Loss:', 64.99528855085373)
('Epoch:', 8, 'Loss:', 31.266997564584017, 'Test Loss:', 64.47643724828959)
('Epoch:', 9, 'Loss:', 30.232711946591735, 'Test Loss:', 63.88381798565388)
('Epoch:', 10, 'Loss:', 29.51552465558052, 'Test Loss:', 65.2800731509924)
Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 1400)
CRIM evaluation:
MRR: 0.10304
P@1: 0.07138
P@5: 0.04817
P@10: 0.04453

#### 24 Projections; Second Cycle
('Epochs: ', 5, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 24, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None')

('Epoch:', 1, 'Loss:', 13.566975106863538, 'Test Loss:', 59.5253451615572)
('Epoch:', 2, 'Loss:', 2.032865798302737, 'Test Loss:', 60.16112444549799)
('Epoch:', 3, 'Loss:', 0.46449146703412225, 'Test Loss:', 64.23307839781046)
('Epoch:', 4, 'Loss:', 0.1069298386116202, 'Test Loss:', 68.53682653605938)
('Epoch:', 5, 'Loss:', 0.027284826716368116, 'Test Loss:', 73.16503396630287)


CRIM evaluation:
MRR: 0.30386
P@1: 0.24817
P@5: 0.14445
P@10: 0.13732

---------------------------------------------------------------------------------------------------
### Experiment with single projection but changing: i) Random initialiser (random + identity); ii) 2nd phase keeps trainining Phi; iii) Change Learning Rate of 2nd phase.

#### 1 Projections; First Cycle
('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None')
('Epoch:', 1, 'Loss:', 176.2122738659382, 'Test Loss:', 177.55351921916008)
('Epoch:', 2, 'Loss:', 104.77461278438568, 'Test Loss:', 105.96388858556747)
('Epoch:', 3, 'Loss:', 93.10278041660786, 'Test Loss:', 95.61527897417545)
('Epoch:', 4, 'Loss:', 83.65695556998253, 'Test Loss:', 86.35148683190346)
('Epoch:', 5, 'Loss:', 76.63541767001152, 'Test Loss:', 80.32908068597317)
('Epoch:', 6, 'Loss:', 71.39560843259096, 'Test Loss:', 74.56421269476414)
('Epoch:', 7, 'Loss:', 67.28997546434402, 'Test Loss:', 70.91447427868843)
('Epoch:', 8, 'Loss:', 63.82748632133007, 'Test Loss:', 67.26722575724125)
('Epoch:', 9, 'Loss:', 60.12399164587259, 'Test Loss:', 65.32279951870441)
('Epoch:', 10, 'Loss:', 57.98733665794134, 'Test Loss:', 63.860010385513306)

CRIM evaluation:
MRR: 0.15094
P@1: 0.11474
P@5: 0.0674
P@10: 0.065

#### 24 Projections; Second Cycle
('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None')
('Epoch:', 1, 'Loss:', 37.44226049259305, 'Test Loss:', 62.1908989623189)
('Epoch:', 2, 'Loss:', 14.947778060100973, 'Test Loss:', 74.91842666268349)
('Epoch:', 3, 'Loss:', 7.418942770687863, 'Test Loss:', 98.69380746781826)
('Epoch:', 4, 'Loss:', 4.618407022906467, 'Test Loss:', 120.58737960457802)
('Epoch:', 5, 'Loss:', 2.871887466167209, 'Test Loss:', 142.8387492597103)
('Epoch:', 6, 'Loss:', 1.9543396250010119, 'Test Loss:', 163.086307823658)
('Epoch:', 7, 'Loss:', 1.3212263471978076, 'Test Loss:', 183.57101076841354)
('Epoch:', 8, 'Loss:', 0.9992299735413326, 'Test Loss:', 209.59179404377937)
('Epoch:', 9, 'Loss:', 0.8104324492987871, 'Test Loss:', 228.58731454610825)
('Epoch:', 10, 'Loss:', 0.5762173827174593, 'Test Loss:', 254.7513089776039)

CRIM evaluation:
MRR: 0.27074
P@1: 0.22148
P@5: 0.13155
P@10: 0.12557


* Overfit manifests for sure.  Person, work-of-art feature heavily as hypernyms even for query terms which are unrelated

### Experiment with dual projections.  Extended epochs of first cycle to 25.  Reduced learning rate of second cycle to reduce overfit and training for 5 epochs

#### First Cycle
('Epochs: ', 25, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 2, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.35, 'Kernel constraint: ', 'None')
('Epoch:', 1, 'Loss:', 160.74599489569664, 'Test Loss:', 160.929179251194)
('Epoch:', 2, 'Loss:', 96.68699912726879, 'Test Loss:', 96.50307583808899)
('Epoch:', 3, 'Loss:', 82.84739479422569, 'Test Loss:', 82.26891721785069)
('Epoch:', 4, 'Loss:', 73.6079184487462, 'Test Loss:', 72.73883473873138)
('Epoch:', 5, 'Loss:', 67.65975984185934, 'Test Loss:', 67.05146090686321)
('Epoch:', 6, 'Loss:', 63.158750265836716, 'Test Loss:', 62.18984428048134)
('Epoch:', 7, 'Loss:', 59.12502943724394, 'Test Loss:', 59.458139173686504)
('Epoch:', 8, 'Loss:', 55.11561170220375, 'Test Loss:', 56.17893383651972)
('Epoch:', 9, 'Loss:', 52.53363721072674, 'Test Loss:', 54.554423585534096)
('Epoch:', 10, 'Loss:', 50.518984742462635, 'Test Loss:', 53.33502481132746)
('Epoch:', 11, 'Loss:', 48.320613726973534, 'Test Loss:', 51.69379674643278)
('Epoch:', 12, 'Loss:', 46.352201879024506, 'Test Loss:', 51.4660424888134)
('Epoch:', 13, 'Loss:', 45.39920901507139, 'Test Loss:', 51.60422394424677)
('Epoch:', 14, 'Loss:', 44.41912394762039, 'Test Loss:', 50.1107277572155)
('Epoch:', 15, 'Loss:', 43.92356888204813, 'Test Loss:', 50.16625649482012)
('Epoch:', 16, 'Loss:', 42.618239261209965, 'Test Loss:', 49.68374668061733)
('Epoch:', 17, 'Loss:', 41.93088800087571, 'Test Loss:', 49.52678156644106)
('Epoch:', 18, 'Loss:', 42.28301604837179, 'Test Loss:', 49.41834541410208)
('Epoch:', 19, 'Loss:', 41.13799152523279, 'Test Loss:', 48.8290878534317)
('Epoch:', 20, 'Loss:', 40.83357220888138, 'Test Loss:', 48.22099205851555)
('Epoch:', 21, 'Loss:', 40.20593152567744, 'Test Loss:', 48.0087883323431)
('Epoch:', 22, 'Loss:', 40.49513000249863, 'Test Loss:', 47.994462229311466)
('Epoch:', 23, 'Loss:', 39.6955735757947, 'Test Loss:', 48.454381965100765)
('Epoch:', 24, 'Loss:', 39.38141195476055, 'Test Loss:', 48.52395910024643)
('Epoch:', 25, 'Loss:', 39.834498304873705, 'Test Loss:', 48.30932606011629)

CRIM evaluation:
MRR: 0.11931
P@1: 0.07872
P@5: 0.05491
P@10: 0.05259

#### Second cycle
('Epochs: ', 5, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 2, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.35, 'Kernel constraint: ', 'None')
('Epoch:', 1, 'Loss:', 30.962760228663683, 'Test Loss:', 46.86121924221516)
('Epoch:', 2, 'Loss:', 18.49303602334112, 'Test Loss:', 43.47840115427971)
('Epoch:', 3, 'Loss:', 12.296762353973463, 'Test Loss:', 41.97538521140814)
('Epoch:', 4, 'Loss:', 8.417490374995396, 'Test Loss:', 40.455049715936184)
('Epoch:', 5, 'Loss:', 6.027016263862606, 'Test Loss:', 40.4208921790123)

CRIM evaluation:
MRR: 0.30662
P@1: 0.2515
P@5: 0.14699
P@10: 0.13961

-------------------------------------------------------------------------------------------------
### Experiment with two Phi layers. 
#### I insert dropout after the hyponym and hypernym embeddings and after the second phi.  Training goes on for 25 epochs.  The solution converges much more slowly than if were using a single affine layer.

('Epochs: ', 25, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.35, 'Kernel constraint: ', 'None')
('Epoch:', 1, 'Loss:', 132.44914108514786, 'Test Loss:', 129.82295709848404)
('Epoch:', 2, 'Loss:', 95.7211739718914, 'Test Loss:', 92.90788823366165)
('Epoch:', 3, 'Loss:', 93.52208907902241, 'Test Loss:', 90.88466887176037)
('Epoch:', 4, 'Loss:', 91.18207442760468, 'Test Loss:', 89.7815543860197)
('Epoch:', 5, 'Loss:', 88.62042617797852, 'Test Loss:', 87.20721289515495)
('Epoch:', 6, 'Loss:', 85.08720774948597, 'Test Loss:', 83.62825165688992)
('Epoch:', 7, 'Loss:', 82.6170591711998, 'Test Loss:', 79.72680546343327)
('Epoch:', 8, 'Loss:', 78.23334312438965, 'Test Loss:', 74.8099833726883)
('Epoch:', 9, 'Loss:', 76.347172498703, 'Test Loss:', 72.44513177871704)
('Epoch:', 10, 'Loss:', 74.18829296529293, 'Test Loss:', 69.96501626074314)
('Epoch:', 11, 'Loss:', 72.2932140827179, 'Test Loss:', 67.46124893426895)
('Epoch:', 12, 'Loss:', 71.2011769413948, 'Test Loss:', 65.32345585525036)
('Epoch:', 13, 'Loss:', 70.07241632044315, 'Test Loss:', 63.981889829039574)
('Epoch:', 14, 'Loss:', 68.04309992492199, 'Test Loss:', 62.387570425868034)
('Epoch:', 15, 'Loss:', 67.2650830000639, 'Test Loss:', 61.06334821879864)
('Epoch:', 16, 'Loss:', 67.25092969834805, 'Test Loss:', 60.93150553107262)
('Epoch:', 17, 'Loss:', 66.12663190811872, 'Test Loss:', 60.26040391623974)
('Epoch:', 18, 'Loss:', 64.85526475310326, 'Test Loss:', 59.32122567296028)
('Epoch:', 19, 'Loss:', 64.72251350432634, 'Test Loss:', 58.52266174554825)
('Epoch:', 20, 'Loss:', 63.96191988885403, 'Test Loss:', 57.419067934155464)
('Epoch:', 21, 'Loss:', 62.82784986868501, 'Test Loss:', 56.976065531373024)
('Epoch:', 22, 'Loss:', 62.8604651093483, 'Test Loss:', 57.06210967898369)
('Epoch:', 23, 'Loss:', 62.14057156443596, 'Test Loss:', 56.878740444779396)
('Epoch:', 24, 'Loss:', 61.323521822690964, 'Test Loss:', 56.8377401381731)
('Epoch:', 25, 'Loss:', 61.14446556568146, 'Test Loss:', 56.42692677676678)

* Having one than one linear layer does not increase the hypothesis space of the model.
* Attempting model with 2 hidden layer each with non-linear activation functions stopped impeded the model from learning anything


-----------------------------------------------------------------------------------------------------
### Testing different batch sizes
('Epochs: ', 8, 'Batch size: ', 1, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.001, 'Bagging:', False)
('Epoch:', 1, 'Loss:', 3066.596051596105, 'Test Loss:', 1.196591, 'MRR:', 0.06733, 'Test accuracy:', 0.365)
('Epoch:', 2, 'Loss:', 2044.704427516088, 'Test Loss:', 1.1396255, 'MRR:', 0.07917, 'Test accuracy:', 0.53)
('Epoch:', 3, 'Loss:', 1773.883358555846, 'Test Loss:', 1.118648, 'MRR:', 0.08222, 'Test accuracy:', 0.59)
('Epoch:', 4, 'Loss:', 1685.0154238501564, 'Test Loss:', 1.2257615, 'MRR:', 0.08352, 'Test accuracy:', 0.57)
('Epoch:', 5, 'Loss:', 1624.6034373817965, 'Test Loss:', 1.2055718, 'MRR:', 0.10333, 'Test accuracy:', 0.57)
('Epoch:', 6, 'Loss:', 1582.9557659688871, 'Test Loss:', 1.2653193, 'MRR:', 0.08917, 'Test accuracy:', 0.61)
('Epoch:', 7, 'Loss:', 1559.463746579131, 'Test Loss:', 1.146983, 'MRR:', 0.08167, 'Test accuracy:', 0.64)
('Epoch:', 8, 'Loss:', 1533.567981993081, 'Test Loss:', 1.1859033, 'MRR:', 0.06533, 'Test accuracy:', 0.625)
Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 1400)
CRIM evaluation:
MRR: 0.10216
MAP: 0.04669
P@1: 0.06738
P@5: 0.04474
P@10: 0.04341

* Perhaps results will improve if training rate is reduced?


# Modifying CRIM to work with multiple phi but send only highest similarity to prediction layer

('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 10, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.001)
('Epoch:', 1, 'Loss:', 192.27924165129662, 'Test Loss:', 185.294663220644)
('Epoch:', 2, 'Loss:', 111.60149729251862, 'Test Loss:', 107.93660750985146)
('Epoch:', 3, 'Loss:', 96.55376133322716, 'Test Loss:', 95.19647890329361)
('Epoch:', 4, 'Loss:', 82.45174576342106, 'Test Loss:', 79.94356182217598)
('Epoch:', 5, 'Loss:', 73.00302350521088, 'Test Loss:', 69.28130987286568)
('Epoch:', 6, 'Loss:', 66.95412650704384, 'Test Loss:', 62.31023855507374)
('Epoch:', 7, 'Loss:', 61.48678085207939, 'Test Loss:', 57.86816193163395)
('Epoch:', 8, 'Loss:', 58.002911515533924, 'Test Loss:', 54.93470601737499)
('Epoch:', 9, 'Loss:', 54.53545202314854, 'Test Loss:', 52.65876457095146)
('Epoch:', 10, 'Loss:', 51.42167618870735, 'Test Loss:', 51.305756598711014)

CRIM evaluation:
MRR: 0.14499
MAP: 0.06773
P@1: 0.10207
P@5: 0.06518
P@10: 0.06227

--------------------------------------------------------------------
('Epochs: ', 13, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 20, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.001)
('Epoch:', 1, 'Loss:', 177.05030685663223, 'Test Loss:', 1.9216485023498535, 'MRR:', 0.0)
('Epoch:', 2, 'Loss:', 105.13821250200272, 'Test Loss:', 2.0207560062408447, 'MRR:', 0.0)
('Epoch:', 3, 'Loss:', 96.8861652314663, 'Test Loss:', 1.8185327053070068, 'MRR:', 0.00667)
('Epoch:', 4, 'Loss:', 89.28570778667927, 'Test Loss:', 1.6428419351577759, 'MRR:', 0.06)
('Epoch:', 5, 'Loss:', 82.289546713233, 'Test Loss:', 1.4797453880310059, 'MRR:', 0.074)
('Epoch:', 6, 'Loss:', 76.45523698627949, 'Test Loss:', 1.336097002029419, 'MRR:', 0.10286)
('Epoch:', 7, 'Loss:', 71.0555507093668, 'Test Loss:', 1.2085713148117065, 'MRR:', 0.09667)
('Epoch:', 8, 'Loss:', 65.79279552400112, 'Test Loss:', 1.1283133029937744, 'MRR:', 0.122)
('Epoch:', 9, 'Loss:', 61.6914467215538, 'Test Loss:', 1.0555990934371948, 'MRR:', 0.13286)
('Epoch:', 10, 'Loss:', 57.9002415984869, 'Test Loss:', 0.9768383502960205, 'MRR:', 0.12952)
('Epoch:', 11, 'Loss:', 54.238177113235, 'Test Loss:', 0.9210295081138611, 'MRR:', 0.117)
('Epoch:', 12, 'Loss:', 51.53789134323597, 'Test Loss:', 0.8832364082336426, 'MRR:', 0.11722)
('Epoch:', 13, 'Loss:', 48.59779189527035, 'Test Loss:', 0.8497580885887146, 'MRR:', 0.11)

CRIM evaluation:
MRR: 0.14087
MAP: 0.06261
P@1: 0.11007
P@5: 0.05931
P@10: 0.05632

-------------------------------------------------------------------------------------------------------
('Epochs: ', 9, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 20, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.001)
('Epoch:', 1, 'Loss:', 218.10481524467468, 'Test Loss:', 1.2180428504943848, 'MRR:', 0.0)
('Epoch:', 2, 'Loss:', 123.68451875448227, 'Test Loss:', 2.3275296688079834, 'MRR:', 0.0)
('Epoch:', 3, 'Loss:', 98.06867711246014, 'Test Loss:', 1.8287553787231445, 'MRR:', 0.06)
('Epoch:', 4, 'Loss:', 82.21024569869041, 'Test Loss:', 1.4681575298309326, 'MRR:', 0.11667)
('Epoch:', 5, 'Loss:', 71.17102481424809, 'Test Loss:', 1.2787588834762573, 'MRR:', 0.099)
('Epoch:', 6, 'Loss:', 64.6261182948947, 'Test Loss:', 1.1748896837234497, 'MRR:', 0.11852)
('Epoch:', 7, 'Loss:', 59.59312414377928, 'Test Loss:', 1.0992566347122192, 'MRR:', 0.09789)
('Epoch:', 8, 'Loss:', 56.39183371514082, 'Test Loss:', 1.0781883001327515, 'MRR:', 0.12233)
('Epoch:', 9, 'Loss:', 52.29561498016119, 'Test Loss:', 1.0466694831848145, 'MRR:', 0.10833)

CRIM evaluation:
MRR: 0.16512
MAP: 0.08061
P@1: 0.12408
P@5: 0.0774
P@10: 0.07424

* After embeddings tuning
('Epochs: ', 5, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 20, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.0005)

('Epoch:', 1, 'Loss:', 29.043478064239025, 'Test Loss:', 0.7365559339523315, 'MRR:', 0.22952)
('Epoch:', 2, 'Loss:', 11.500816403888166, 'Test Loss:', 0.5949599742889404, 'MRR:', 0.22786)
('Epoch:', 3, 'Loss:', 5.905871827970259, 'Test Loss:', 0.5205121040344238, 'MRR:', 0.22167)
('Epoch:', 4, 'Loss:', 3.320489032255864, 'Test Loss:', 0.4699397385120392, 'MRR:', 0.22119)
('Epoch:', 5, 'Loss:', 2.18644647533074, 'Test Loss:', 0.4323105216026306, 'MRR:', 0.23333)

CRIM evaluation:
MRR: 0.27001
MAP: 0.12721
P@1: 0.23215
P@5: 0.119
P@10: 0.11395

* Changed 2nd phase parameters
('Epochs: ', 7, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 20, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.001)
('Epoch:', 1, 'Loss:', 22.901207104325294, 'Test Loss:', 0.6052702069282532, 'MRR:', 0.22352)
('Epoch:', 2, 'Loss:', 5.384110522107221, 'Test Loss:', 0.4855782985687256, 'MRR:', 0.22119)
('Epoch:', 3, 'Loss:', 2.196897844092746, 'Test Loss:', 0.4189225435256958, 'MRR:', 0.20286)
('Epoch:', 4, 'Loss:', 1.1273050970230543, 'Test Loss:', 0.37064453959465027, 'MRR:', 0.19952)
('Epoch:', 5, 'Loss:', 0.5745529867244841, 'Test Loss:', 0.3379283845424652, 'MRR:', 0.18952)
('Epoch:', 6, 'Loss:', 0.3683943189241745, 'Test Loss:', 0.3203800916671753, 'MRR:', 0.19952)
('Epoch:', 7, 'Loss:', 0.21765478086467738, 'Test Loss:', 0.3079485595226288, 'MRR:', 0.19952)
CRIM evaluation:
MRR: 0.24346
MAP: 0.11254
P@1: 0.1988
P@5: 0.10505
P@10: 0.10311


---------------------------------------------------------------------------------------------------
* In this section, I normalise the dot product of the hyponym projection and hypernym embedding.
* Performance improves significantly, even with 1 cluster:

('Epochs: ', 13, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.001)
('Epoch:', 1, 'Loss:', 216.5801951289177, 'Test Loss:', 1.01119065284729, 'MRR:', 0.0)
('Epoch:', 2, 'Loss:', 166.6781222820282, 'Test Loss:', 1.3387771844863892, 'MRR:', 0.0)
('Epoch:', 3, 'Loss:', 134.3004250228405, 'Test Loss:', 1.669001579284668, 'MRR:', 0.0)
('Epoch:', 4, 'Loss:', 115.40679916739464, 'Test Loss:', 1.9242345094680786, 'MRR:', 0.02)
('Epoch:', 5, 'Loss:', 105.27983355522156, 'Test Loss:', 1.9711346626281738, 'MRR:', 0.04667)
('Epoch:', 6, 'Loss:', 99.74403658509254, 'Test Loss:', 1.8888331651687622, 'MRR:', 0.10082)
('Epoch:', 7, 'Loss:', 96.84595969319344, 'Test Loss:', 1.785994291305542, 'MRR:', 0.08582)
('Epoch:', 8, 'Loss:', 94.51537863910198, 'Test Loss:', 1.7007386684417725, 'MRR:', 0.09222)
('Epoch:', 9, 'Loss:', 92.27100689709187, 'Test Loss:', 1.6312320232391357, 'MRR:', 0.09552)
('Epoch:', 10, 'Loss:', 90.04288397729397, 'Test Loss:', 1.5578685998916626, 'MRR:', 0.10834)
('Epoch:', 11, 'Loss:', 88.05075532197952, 'Test Loss:', 1.4859631061553955, 'MRR:', 0.11169)
('Epoch:', 12, 'Loss:', 85.85826447606087, 'Test Loss:', 1.4254134893417358, 'MRR:', 0.108)
('Epoch:', 13, 'Loss:', 84.02545890212059, 'Test Loss:', 1.3709036111831665, 'MRR:', 0.10817)

CRIM evaluation:
MRR: 0.17462
MAP: 0.08124
P@1: 0.13809
P@5: 0.07732
P@10: 0.07283

* We tune embeddings for only 2 epochs
('Epochs: ', 2, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.0005)

('Epoch:', 1, 'Loss:', 65.86931462585926, 'Test Loss:', 1.1016316413879395, 'MRR:', 0.27186)
('Epoch:', 2, 'Loss:', 45.293831795454025, 'Test Loss:', 0.8902890682220459, 'MRR:', 0.28783)

CRIM evaluation:
MRR: 0.29305
MAP: 0.14925
P@1: 0.24883
P@5: 0.14403
P@10: 0.13475

* Tuning for a futher two epochs improves the validation MRR, but test metric deterioriate 

('Epochs: ', 2, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.0005)
('Epoch:', 1, 'Loss:', 32.49462116137147, 'Test Loss:', 0.7281622290611267, 'MRR:', 0.284)
('Epoch:', 2, 'Loss:', 23.98638728633523, 'Test Loss:', 0.6015497446060181, 'MRR:', 0.30733)

CRIM evaluation:
MRR: 0.25017
MAP: 0.13469
P@1: 0.1968
P@5: 0.13179
P@10: 0.12452

* Performance decreases the more epochs we add despite the fact that both training loss and test loss keep decreasing.  Validation MRR is a more important training metric to keep track of.

('Epochs: ', 2, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.0005)
('Epoch:', 1, 'Loss:', 18.0652209892869, 'Test Loss:', 0.5042541027069092, 'MRR:', 0.274)
('Epoch:', 2, 'Loss:', 13.913509592413902, 'Test Loss:', 0.4291400909423828, 'MRR:', 0.27622)

MRR: 0.22766
MAP: 0.12481
P@1: 0.17078
P@5: 0.12266
P@10: 0.11679

* Fine-tuning for just 1 epoch:

('Epochs: ', 1, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.0005)
('Epoch:', 1, 'Loss:', 65.84822230041027, 'Test Loss:', 1.1002403497695923, 'MRR:', 0.25405)

MRR: 0.29843
MAP: 0.14871
P@1: 0.25751
P@5: 0.14107
P@10: 0.13354

* Now we keep embeddings frozen and we traing the phi layer for a few more epochs

('Epochs: ', 5, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.001)
('Epoch:', 1, 'Loss:', 57.45014962553978, 'Test Loss:', 0.9213581681251526, 'MRR:', 0.28186)
('Epoch:', 2, 'Loss:', 54.33015509694815, 'Test Loss:', 0.8853888511657715, 'MRR:', 0.28352)
('Epoch:', 3, 'Loss:', 51.75078434497118, 'Test Loss:', 0.8243922591209412, 'MRR:', 0.2855)
('Epoch:', 4, 'Loss:', 49.316737711429596, 'Test Loss:', 0.7884426712989807, 'MRR:', 0.28686)
('Epoch:', 5, 'Loss:', 47.065613731741905, 'Test Loss:', 0.7453060746192932, 'MRR:', 0.30852)

MRR: 0.30175
MAP: 0.15109
P@1: 0.25751
P@5: 0.14383
P@10: 0.13604

* And a further 5 epochs
('Epochs: ', 5, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 1, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.001)
('Epoch:', 1, 'Loss:', 44.93913523107767, 'Test Loss:', 0.7189047932624817, 'MRR:', 0.28852)
('Epoch:', 2, 'Loss:', 42.82094585895538, 'Test Loss:', 0.7057857513427734, 'MRR:', 0.28786)
('Epoch:', 3, 'Loss:', 41.27894340455532, 'Test Loss:', 0.6507096886634827, 'MRR:', 0.27452)
('Epoch:', 4, 'Loss:', 39.19503653794527, 'Test Loss:', 0.613694965839386, 'MRR:', 0.27952)
('Epoch:', 5, 'Loss:', 37.61957763880491, 'Test Loss:', 0.623578667640686, 'MRR:', 0.27952)

CRIM evaluation:
MRR: 0.30639
MAP: 0.15227
P@1: 0.26217
P@5: 0.14563
P@10: 0.13702

----------------------------------------------------------------------------------------------------
* In this experiment, we train a 10 projection model in three phases.

('Epochs: ', 10, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 10, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.001)
('Epoch:', 1, 'Loss:', 223.62342816591263, 'Test Loss:', 0.9674524664878845, 'MRR:', 0.0)
('Epoch:', 2, 'Loss:', 166.71091252565384, 'Test Loss:', 1.291346788406372, 'MRR:', 0.005)
('Epoch:', 3, 'Loss:', 130.67713418602943, 'Test Loss:', 1.6250698566436768, 'MRR:', 0.0)
('Epoch:', 4, 'Loss:', 110.79544314742088, 'Test Loss:', 1.903112769126892, 'MRR:', 0.03167)
('Epoch:', 5, 'Loss:', 101.37133565545082, 'Test Loss:', 2.0412981510162354, 'MRR:', 0.06067)
('Epoch:', 6, 'Loss:', 97.36877712607384, 'Test Loss:', 1.931593656539917, 'MRR:', 0.06357)
('Epoch:', 7, 'Loss:', 94.5494154393673, 'Test Loss:', 1.8447002172470093, 'MRR:', 0.074)
('Epoch:', 8, 'Loss:', 91.85252003371716, 'Test Loss:', 1.7642910480499268, 'MRR:', 0.103)
('Epoch:', 9, 'Loss:', 89.23252408206463, 'Test Loss:', 1.6848324537277222, 'MRR:', 0.13833)
('Epoch:', 10, 'Loss:', 86.65304459631443, 'Test Loss:', 1.6097363233566284, 'MRR:', 0.144)

CRIM evaluation:
MRR: 0.17335
MAP: 0.07732
P@1: 0.14076
P@5: 0.07268
P@10: 0.06865

* Second phase:
('Epochs: ', 2, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 10, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.0005)
('Epoch:', 1, 'Loss:', 71.49462106823921, 'Test Loss:', 1.3946821689605713, 'MRR:', 0.19686)
('Epoch:', 2, 'Loss:', 57.22928865253925, 'Test Loss:', 1.2148321866989136, 'MRR:', 0.22533)

CRIM evaluation:
MRR: 0.28018
MAP: 0.14123
P@1: 0.23749
P@5: 0.1342
P@10: 0.12747


* In third phase, I attempted to train embeddings and phi together
('Epochs: ', 2, 'Batch size: ', 32, 'm: ', 10, 'pki_k: ', 10, 'train_embeddings: ', False, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'Dropout rate: ', 0.3, 'Kernel constraint: ', 'None', 'Learning rate: ', 0.001)
('Epoch:', 1, 'Loss:', 51.64020486921072, 'Test Loss:', 0.8209819197654724, 'MRR:', 0.25533)
('Epoch:', 2, 'Loss:', 37.853846058249474, 'Test Loss:', 0.6199454665184021, 'MRR:', 0.28622)

CRIM evaluation:
MRR: 0.24635
MAP: 0.12464
P@1: 0.19813
P@5: 0.11927
P@10: 0.1132

Results were actually worse

# Hard Clustering

## Common Routines

In [ ]:
# light-weight data class containing only 
class ClusterHybrid:
            
    def __init__(self, cluster, cluster_predictions,
                 orig_data,                 
                 phi_init, sigmoid_kernel_constraint,
                 embeddings_layer, dropout_rate, learning_rate
                 ):
        
        # create model
        self.model = self._init_model(phi_init=phi_init,                                       
                                      embeddings_layer = embedding_layer, 
                                      embeddings_dim = orig_data.embeddings_dim,
                                      sigmoid_kernel_constraint = sigmoid_kernel_constraint,
                                      dropout_rate = dropout_rate, learning_rate = learning_rate)
        
        self.cluster_id = cluster
        self.tokenizer = orig_data.tokenizer
        
        self.train_query = map(lambda x: orig_data.train_query[x], np.where(cluster_predictions == cluster)[0])
        self.train_hyper = map(lambda x: orig_data.train_hyper[x], np.where(cluster_predictions == cluster)[0])
                                
        self.valid_query = orig_data.valid_query
        self.valid_hyper = orig_data.valid_hyper
        
        self.train_query_seq, self.train_hyper_seq, self.valid_query_seq, self.valid_hyper_seq =\
        map(lambda x: orig_data.tokenizer.texts_to_sequences(x),\
            [self.train_query, self.train_hyper, self.valid_query, self.valid_hyper])
        
        self.train_query_seq, self.train_hyper_seq, self.valid_query_seq, self.valid_hyper_seq =\
        map(lambda x: np.asarray(x, dtype='int32'),\
            [self.train_query_seq, self.train_hyper_seq, self.valid_query_seq, self.valid_hyper_seq])
        

        self.loss = 0.
        self.test_loss = 0
    
    def _init_model(self, phi_init, sigmoid_kernel_constraint, 
                    embeddings_layer, embeddings_dim,
                    dropout_rate, learning_rate):
                                                   
        return get_CRIM_model(phi_k = phi_init, 
                              train_embeddings = false,
                              embeddings_dim = embeddings_dim,
                              embeddings_layer = embeddings_layer,
                              phi_init = random_identity,
                              sigmoid_kernel_constraint = sigmoid_kernel_constraint,
                              dropout_rate = dropout_rate,
                              learning_rate = learning_rate)
        
    
    def update_loss(self, new_loss):
        self.loss += new_loss
        
    def update_test_loss(self, new_loss):
        self.test_loss += new_loss   

## Hard Clustering Specific

In [ ]:
from sklearn.cluster import KMeans

cluster_k = 5

In [ ]:
# calculate offsets 
training_query_vector = np.zeros((len(data.train_query), data.embeddings_dim))
training_hyper_vector = np.zeros((len(data.train_hyper), data.embeddings_dim))
for i in range(len(data.train_query)):
    query_embedding_lookup = data.tokenizer.word_index[data.train_query[i]]
    hyper_embedding_lookup = data.tokenizer.word_index[data.train_hyper[i]]
    
    training_query_vector[i] = data.embedding_matrix[query_embedding_lookup]
    training_hyper_vector[i] = data.embedding_matrix[hyper_embedding_lookup]
    

train_offsets = training_hyper_vector - training_query_vector

In [ ]:
km = KMeans(n_clusters = cluster_k, n_jobs=-1, random_state=42)
#km.fit_predict(train_offsets)
km.fit_predict(train_offsets)

In [ ]:
from collections import Counter
Counter(km.predict(train_offsets))

In [ ]:
#np.where(km.predict(train_offsets) == 0)
map(lambda x: (data.train_query[x], data.train_hyper[x]), np.where(km.predict(train_offsets) == 4)[0])

In [ ]:
# implement training algorithm modification to deal with clusters
def train_on_clusters(models,      # the clustered models on which parameters will be learnt
                      epochs,      # number of epochs to run          
                      batch_size,  # size of mini-batch 
                      m,           # number of negative samples
                      #data,        # data required for training                              
                      neg_strategy
                     ):                                   
    
    # train algorithm
    for epoch in range(epochs):
        # train each model on their corresponding data
        for model in models:            
            print ("Doing model: ", model.cluster_id, "; epoch: ", epoch)
            # reset loss
            model.loss = 0.
            model.test_loss = 0.
            
            samples = np.arange(len(model.train_query_seq))
            validation_samples = np.arange(len(model.valid_query_seq))
                                        
            np.random.shuffle(samples)                    
                        
            for b in range(0, len(samples), batch_size):    
                if ((b + 1) % 500) == 0:
                    print ('Model: ', model.cluster_id, '; processed ', idx+1, 'samples.')
                                    
                batch_X_term, batch_X_hyper, batch_y_label =\
                    extend_batch_with_negatives(model.train_query_seq[b:b + batch_size], 
                                                model.train_hyper_seq[b:b + batch_size],
                                                neg_strategy,
                                                model.tokenizer, m
                                               )  

                #print model.cluster_id, len(batch_X_term)
                model.update_loss(model.model.train_on_batch([batch_X_term, batch_X_hyper], batch_y_label)[0])                                
                
        # validate on entire validation set after training each model
            
        test_query, test_hyper, test_y_label =\
            extend_batch_with_negatives(model.valid_query_seq, 
                                        model.valid_hyper_seq,
                                        neg_strategy,
                                        model.tokenizer, m
                                       )  
        #batch_label = [1.] * batch_query.shape[0]
        for q, h, l in zip(test_query, test_hyper, test_y_label):                                    
            test_losses = list(map(lambda c: c.model.test_on_batch([q, h], [l])[0], models))
            best_cluster = np.argmin(test_losses)
            models[best_cluster].update_test_loss(test_losses[best_cluster])
                            
        print('Epoch:', epoch+1,\
              'Loss:', np.mean([model.loss for model in models]),\
              'Test Loss:', np.mean([model.test_loss for model in models]))    

In [ ]:
# initialise clusters
embeddings_layer = get_embeddings_model(dim=data.embeddings_dim, embedding_matrix=data.embedding_matrix)

cluster_list = []
cluster_predictions = km.predict(train_offsets)



for c in range(cluster_k):
    cluster_list.append(ClusterHybrid(cluster=c, 
                                      cluster_predictions = cluster_predictions, 
                                      orig_data = data, 
                                      phi_init = random_identity, 
                                      sigmoid_kernel_constraint = None,
                                      embeddings_layer = embeddings_layer,
                                      dropout_rate = 0.3, 
                                      learning_rate = 0.001 ))

In [ ]:
# negative sampling options
neg_sampling_options = {'synonym':data.synonyms,                                                 
                        'random':data.random_words
                       }

# phi random init options
phi_init_options = {'random_plus_identity': random_plus_identity,
                    'random_identity': random_identity, 
                    'random_normal': random_normal}

kernel_constraints = {'None': None, 'ForceToOne': ForceToOne()}

# positive batch size
batch_size = 32

# implement mini-batch stochastic training
epochs = 10


# number of negative samples
m = 1
# number of projections
phi_k = 1
# train (True) or freeze
train_embeddings = False
# negative sample strategy
negative_option = 'random'
# initialise phi strategy
phi_init_option = 'random_identity'
# constrain LR parameter
kernel_constraint_option = 'None'
# dropout rate
dropout_rate = 0.3
learning_rate = 0.001


print ("Start training")
train_on_clusters(cluster_list, epochs, batch_size, m, neg_sampling_options[negative_option])

In [ ]:
print ("Generating predictions...")
cluster_predictions = predict_cluster_hypernyms(data, cluster_list)

print ("CRIM evaluation:")
score_names, all_scores = get_evaluation_scores((data.test_query, data.test_hyper), cluster_predictions)
for k in range(len(score_names)):
    print (score_names[k]+': '+str(round(sum([score_list[k] for score_list in all_scores]) / len(all_scores), 5)))


In [ ]:
#cluster_list[0].model.get_layer(name='Phi0').get_weights()[0]

for idx, m in enumerate(clusters):
    print idx, m.loss, m.test_loss, m.model.get_layer(name='Prediction').get_weights()[0]

i = data.tokenizer.word_index['tussle']
j = data.tokenizer.word_index['student']
for c in clusters:
    print c.model.predict([[i], [j]])[0]

## Hard Clustering Results

* batch_size = 32
* epochs = 10
* m = 1
* phi_k = 1
* train_embeddings = False
* negative_option = 'random'
* phi_init_option = 'random_identity'
* kernel_constraint_option = 'None'
* dropout_rate = 0.3
* learning_rate = 0.001

* Performed hard clustering using k = 5.  Vector offset used as basis for clustering.

Start training
('Doing model: ', 0, '; epoch: ', 0)
('Doing model: ', 1, '; epoch: ', 0)
('Doing model: ', 2, '; epoch: ', 0)
('Doing model: ', 3, '; epoch: ', 0)
('Doing model: ', 4, '; epoch: ', 0)
('Epoch:', 1, 'Loss:', 51.20275011062622, 'Test Loss:', 54.16907963752747)
('Doing model: ', 0, '; epoch: ', 1)
('Doing model: ', 1, '; epoch: ', 1)
('Doing model: ', 2, '; epoch: ', 1)
('Doing model: ', 3, '; epoch: ', 1)
('Doing model: ', 4, '; epoch: ', 1)
('Epoch:', 2, 'Loss:', 49.9156032204628, 'Test Loss:', 51.34425748586655)
('Doing model: ', 0, '; epoch: ', 2)
('Doing model: ', 1, '; epoch: ', 2)
('Doing model: ', 2, '; epoch: ', 2)
('Doing model: ', 3, '; epoch: ', 2)
('Doing model: ', 4, '; epoch: ', 2)
('Epoch:', 3, 'Loss:', 47.629679238796236, 'Test Loss:', 47.31288024187088)
('Doing model: ', 0, '; epoch: ', 3)
('Doing model: ', 1, '; epoch: ', 3)
('Doing model: ', 2, '; epoch: ', 3)
('Doing model: ', 3, '; epoch: ', 3)
('Doing model: ', 4, '; epoch: ', 3)
('Epoch:', 4, 'Loss:', 44.66189357638359, 'Test Loss:', 42.583502805233)
('Doing model: ', 0, '; epoch: ', 4)
('Doing model: ', 1, '; epoch: ', 4)
('Doing model: ', 2, '; epoch: ', 4)
('Doing model: ', 3, '; epoch: ', 4)
('Doing model: ', 4, '; epoch: ', 4)
('Epoch:', 5, 'Loss:', 41.36941378712654, 'Test Loss:', 37.747225880622864)
('Doing model: ', 0, '; epoch: ', 5)
('Doing model: ', 1, '; epoch: ', 5)
('Doing model: ', 2, '; epoch: ', 5)
('Doing model: ', 3, '; epoch: ', 5)
('Doing model: ', 4, '; epoch: ', 5)
('Epoch:', 6, 'Loss:', 38.09736560583114, 'Test Loss:', 33.176356403529645)
('Doing model: ', 0, '; epoch: ', 6)
('Doing model: ', 1, '; epoch: ', 6)
('Doing model: ', 2, '; epoch: ', 6)
('Doing model: ', 3, '; epoch: ', 6)
('Doing model: ', 4, '; epoch: ', 6)
('Epoch:', 7, 'Loss:', 34.89849599599838, 'Test Loss:', 29.117837768793105)
('Doing model: ', 0, '; epoch: ', 7)
('Doing model: ', 1, '; epoch: ', 7)
('Doing model: ', 2, '; epoch: ', 7)
('Doing model: ', 3, '; epoch: ', 7)
('Doing model: ', 4, '; epoch: ', 7)
('Epoch:', 8, 'Loss:', 32.04033596813679, 'Test Loss:', 25.581615015119315)
('Doing model: ', 0, '; epoch: ', 8)
('Doing model: ', 1, '; epoch: ', 8)
('Doing model: ', 2, '; epoch: ', 8)
('Doing model: ', 3, '; epoch: ', 8)
('Doing model: ', 4, '; epoch: ', 8)
('Epoch:', 9, 'Loss:', 29.56286123096943, 'Test Loss:', 22.568919814378024)
('Doing model: ', 0, '; epoch: ', 9)
('Doing model: ', 1, '; epoch: ', 9)
('Doing model: ', 2, '; epoch: ', 9)
('Doing model: ', 3, '; epoch: ', 9)
('Doing model: ', 4, '; epoch: ', 9)
('Epoch:', 10, 'Loss:', 27.37933742403984, 'Test Loss:', 20.065976665169)


CRIM evaluation:
MRR: 0.03292
P@1: 0.02402
P@5: 0.01587
P@10: 0.01505

---------------------------------------------------------------------------------------------


* We could try to train a simple model for a few epochs.  
* Once that is done we can refine the results by creating clusters, each initialised with the phi weights from the previous attempt

# Soft Clustering

In [ ]:
class YamaneCluster(ClusterHybrid):
    def __init__(self, phi_init, 
                 embeddings_layer, embeddings_dim,
                 sigmoid_kernel_constraint, 
                 dropout_rate, learning_rate):        
        
        # create Keras model
        self.model = self._init_model(phi_init = phi_init, 
                                      embeddings_layer = embeddings_layer,
                                      embeddings_dim = embeddings_dim,
                                      sigmoid_kernel_constraint = sigmoid_kernel_constraint,
                                      dropout_rate  = dropout_rate,
                                      learning_rate = learning_rate)
        # initialise variables     
        self.epoch_count = 0
        self.loss = 0.
        self.test_loss = 0.
        self.mrr = []
    
    def increment_epoch(self):
        self.epoch_count += 1

In [ ]:
def yamane_train(
    epochs,      # number of epochs to run
    m,           # number of negative samples
    data,        # class instance containing all the data required for training/testing        
    embedding_layer,     # shared embeddings layer
    threshold    = 0.15,     # threshold; similarity below this score will trigger new cluster
    negative_option = 'random', # pass dictionary of random terms 
    phi_init_option = None,     # phi dense layer initialisation strategy
    sigmoid_constraint_option = 'None',
    dropout_rate = 0.,
    learning_rate = 0.001,
    cluster_max = 5
): 
    
    phi_init_options = {'random_identity': random_identity, 
                        'random_normal': random_normal, 
                        'random_plus_identity': random_plus_identity}
    neg_sampling_options = {'synonym':data.synonyms, 'random':data.random_words}
    sigmoid_constraint_options = {'ForceToOne': ForceToOne(), 'None': None}
    
    sigmoid_kernel_constraint = sigmoid_constraint_options[sigmoid_constraint_option]
    
    neg_strategy = neg_sampling_options[negative_option]
            
    # create sequences
    # we have two sets of inputs: one for training query and hypernym terms;
    #                             another for the validation query/hyper terms;
    term_train_seq = data.tokenizer.texts_to_sequences(data.train_query)
    hyper_train_seq = data.tokenizer.texts_to_sequences(data.train_hyper)

    #term_test_seq = data.tokenizer.texts_to_sequences(data.valid_query)
    #hyper_test_seq = data.tokenizer.texts_to_sequences(data.valid_hyper)
    
    # convert all to arrays
    #term_train_seq, hyper_train_seq, term_test_seq, hyper_test_seq =\
    #[np.asarray(x, dtype='int32') for x in [term_train_seq, hyper_train_seq, term_test_seq, hyper_test_seq]]
    
    term_train_seq, hyper_train_seq = [np.asarray(x, dtype='int32') for x in [term_train_seq, hyper_train_seq]]
            
    # this list stores which cluster each training sequence pertains to
    sample_clusters = np.zeros(len(term_train_seq), dtype='int32')
    
    print ("m: ", m, "lambda: ", threshold, "max epoch per cluster: ", epochs, 
           "Negative sampling: ", negative_option, "Phi Init: ", phi_init_option,
           "sigmoid_kernel_constraint: ", sigmoid_constraint_option, 
           "dropout: ", dropout_rate, "learning_rate: ", learning_rate, 
           "cluster_max: ", cluster_max          
          )
    
    
    print ("Sample clusters size: ", len(sample_clusters))
    # list containing 1 model per cluster
    clusters = []    
        
    clusters.append(YamaneCluster(phi_init = phi_init_options[phi_init_option],
                                  embeddings_layer = embedding_layer,
                                  embeddings_dim = data.embeddings_dim,
                                  sigmoid_kernel_constraint = sigmoid_kernel_constraint,
                                  dropout_rate = dropout_rate,
                                  learning_rate = learning_rate))
                    
    # get training set indices
    indices = np.arange(len(term_train_seq))  
    
    # get test set indices
    #test_indices = np.arange(len(term_test_seq))
            
    # initialise each training sample to cluster 0
    sample_clusters[indices] = 0        
    
    # seed random generator
    np.random.seed(42)
    
    # indicator of "current" sample cluster index
    z_i = 0
                    
    while np.min([c.epoch_count for c in clusters]) < epochs:
        # reset loss for each cluster                        
        for c in clusters:
            if c.epoch_count < epochs:                
                c.loss = 0.
            c.test_loss = 0.                
        
        # shuffle indices every epoch
        np.random.shuffle(indices)
        
        # train algorithm by stochastic gradient descent, one sample at a time
        # learn 1 matrix of first epoch only
        for idx, i in enumerate(indices):                        
            if (idx + 1) % 1000 == 0:
                print ("Processed ", idx+1, "samples...")
            
            # calculate similarity on all clusters
            sim = list(map(lambda x: x.model.predict([term_train_seq[i], hyper_train_seq[i]]), clusters))            
            max_sim = np.argmax(sim)
            #print "Term:", tokenizer.index_word[term_train_seq[i][0]], 'Hyper:', tokenizer.index_word[hyper_train_seq[i][0]], "Max Similarity cluster:", max_sim, "(sim = %0.8f)" % (sim[max_sim])
            # limit cluster creation to a max of 25.
            if ((clusters[0].epoch_count > 0) and (sim[max_sim] < threshold) and (len(clusters) < cluster_max)): 
                # add new cluster to list of clusters
                print data.tokenizer.index_word[term_train_seq[i][0]], data.tokenizer.index_word[hyper_train_seq[i][0]]
                print max_sim, sim[max_sim]
                clusters.append(YamaneCluster(phi_init = phi_init_options[phi_init_option],
                                               embeddings_layer = embedding_layer,
                                               embeddings_dim = data.embeddings_dim,
                                               sigmoid_kernel_constraint = sigmoid_kernel_constraint,
                                               dropout_rate = dropout_rate,
                                               learning_rate = learning_rate))
                
                # assign current cluster index to latest model
                z_i = len(clusters) - 1
                sample_clusters[i] = z_i
            else:            
                z_i = max_sim
                sample_clusters[i] = z_i                
                        
            # if current cluster reached/exceeded epoch count, skip current sample (i.e don't update cluster)
            if clusters[z_i].epoch_count < epochs:                                            
                # extend samples in cluster with negative samples
                batch_X_term, batch_X_hyper, batch_y_label =\
                    extend_batch_with_negatives(term_train_seq[i], 
                                                hyper_train_seq[i],
                                                neg_strategy,
                                                data.tokenizer, m
                                               )  

                # update parameters of cluster 
                clusters[z_i].update_loss(
                    clusters[z_i].model.train_on_batch([batch_X_term, batch_X_hyper], batch_y_label)[0]
                )
        
            ####################### END OF EPOCH #######################                  
        
        # instead of test loss, measure MRR as a more indicative validation metric
        print ("Running evaluation on trial data set...")
        predictions = predict_cluster_hypernyms(data.valid_query, data.tokenizer, clusters)
        _, all_scores = get_evaluation_scores((data.valid_query, data.valid_hyper), predictions)
        mrr = round(sum([score_list[0] for score_list in all_scores]) / len(all_scores), 5)
        clusters[0].mrr.append(mrr)
        
        # increase epoch count for clusters
        for cluster in clusters:            
            cluster.epoch_count += 1
                
        print('Epoch:', max([c.epoch_count for c in clusters]), 'Cluster #:', len(clusters) ,
              'Loss:', np.mean([c.loss for c in clusters]),
              'Test MRR:', mrr)
    return clusters, sample_clusters

In [ ]:
import datetime

# initialise embedding later which will be shared among all clusters
embedding_layer = get_embeddings_model(dim=data.embeddings_dim, embedding_matrix=data.embedding_matrix)
epochs = 10
m = 10

print ("Training started at: %s" %  (datetime.datetime.now()))
clusters, sample_clusters =\
    yamane_train(epochs, m, 
                 data,
                 embedding_layer,
                 threshold = 0.13,
                 negative_option = 'random',
                 phi_init_option = 'random_plus_identity',
                 sigmoid_constraint_option = 'ForceToOne',
                 dropout_rate = 0.3,
                 learning_rate = 0.001,
                 cluster_max = 30
                )

print ("Training concluded at: %s" % (datetime.datetime.now()))



In [ ]:
for c in clusters:
    print c.loss, c.test_loss, c.epoch_count, c.mrr
    
    
map(lambda x: (data.train_query[x], data.train_hyper[x]), np.where(sample_clusters == 20)[0])
#c15 = Counter(map(lambda x: data.train_hyper[x], np.where(sample_clusters == 18)[0]))
#sorted(c15.items(), key = lambda (k,v): v, reverse=True)
#Counter(sample_clusters)

### Evaluate without attempting to cluster test terms

In [ ]:
print ("Generating predictions...")
yamane_predictions = predict_cluster_hypernyms(data.test_query, data.tokenizer, clusters)

print ("CRIM evaluation:")
score_names, all_scores = get_evaluation_scores((data.test_query, data.test_hyper), yamane_predictions)
for k in range(len(score_names)):
    print (score_names[k]+': '+str(round(sum([score_list[k] for score_list in all_scores]) / len(all_scores), 5)))



In [153]:
predict_cluster_hypernyms(['campo_san_polo'], data.tokenizer, clusters)

#filter(lambda x: x.startswith('campo'), data.tokenizer.word_index.keys())


MemoryError: 

### Train KNN classifier on clustering data jointly learnt by model

In [75]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5, weights='distance')

# prepare knn dataset based on learnt clusters
train_seq = np.array(data.tokenizer.texts_to_sequences(data.train_query))

X_knn = {}
for idx, c in enumerate(clusters):
    cluster_ids = np.where(sample_clusters == idx)
    # we can reduce duplicate terms to unique terms    
    uniq_terms = np.unique(train_seq[cluster_ids])
    #print (uniq_terms)    
    X_knn[idx] = data.embedding_matrix[uniq_terms]  

X_features = X_knn[0]
y = np.zeros(X_knn[0].shape[0], dtype='int16')

for k in range(1,len(clusters)):
    X_features = np.vstack((X_features, X_knn[k]))
    y = np.hstack((y, np.array([k] * X_knn[k].shape[0])))
    
neigh.fit(X_features, y) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')

In [76]:
print ("Generating predictions...")
yamane_predictions = predict_cluster_hypernyms(data.test_query, data.tokenizer, clusters, neigh)

print ("CRIM evaluation:")
score_names, all_scores = get_evaluation_scores((data.test_query, data.test_hyper), yamane_predictions)
for k in range(len(score_names)):
    print (score_names[k]+': '+str(round(sum([score_list[k] for score_list in all_scores]) / len(all_scores), 5)))


Generating predictions...
('Done', 100)
('Done', 200)
('Done', 300)
('Done', 400)
('Done', 500)
('Done', 600)
('Done', 700)
('Done', 800)
('Done', 900)
('Done', 1000)
('Done', 1100)
('Done', 1200)
('Done', 1300)
('Done', 1400)
CRIM evaluation:
MRR: 0.27532
MAP: 0.13254
P@1: 0.23149
P@5: 0.12402
P@10: 0.11837


In [77]:
yamane_predictions
#crim_get_top_hypernyms('turonian', None, clusters[15].model, data, 15)

{u'scouter': [u'person',
  u'movie',
  u'actor',
  u'tv_show',
  u'television_show',
  u'film',
  u'television_series',
  u'writer',
  u'television_program',
  u'tv_program',
  u'musician',
  u'motion_picture',
  u'work_of_art',
  u'tv_series',
  u'feature_film'],
 u'gatekeeper': [u'person',
  u'software',
  u'computer_software',
  u'software_program',
  u'software_package',
  u'software_application',
  u'computer_system',
  u'communication_medium',
  u'company',
  u'enterprise',
  u'applications_software',
  u'computer',
  u'computer_program',
  u'code',
  u'leader'],
 u'mackerel': [u'animal',
  u'plant',
  u'fish',
  u'plant_part',
  u'shellfish',
  u'brine_shrimp',
  u'crustacea',
  u'animal_tissue',
  u'edible',
  u'freshwater_fish',
  u'marine_animal',
  u'waterbody',
  u'algae',
  u'earthworm',
  u'food_web'],
 u'prefix': [u'software',
  u'software_program',
  u'software_package',
  u'computer_software',
  u'computer_program',
  u'computer_code',
  u'software_application',
  u'pe

In [ ]:
from collections import Counter
#yamane_predictions['dashi']
np.mean(clusters[10].model.get_layer(name='Phi0').get_weights()[0])
Counter(sample_clusters)

# Yamane Results

Training started...
* ('m: ', 10, 'lambda: ', 0.15, 'max epoch per cluster: ', 15, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_normal')
('Epoch:', 1, 'Cluster #:', 25, 'Loss:', 185.25700701117515, 'Test Loss:', 462.11729974992573)
('Epoch:', 2, 'Cluster #:', 25, 'Loss:', 94.78926725581289, 'Test Loss:', 257.9614339789539)
('Epoch:', 3, 'Cluster #:', 25, 'Loss:', 68.510555833322, 'Test Loss:', 168.7855049063693)
('Epoch:', 4, 'Cluster #:', 25, 'Loss:', 56.637282415106895, 'Test Loss:', 131.58040585272218)
('Epoch:', 5, 'Cluster #:', 25, 'Loss:', 48.74302359417314, 'Test Loss:', 114.5913256756349)
('Epoch:', 6, 'Cluster #:', 25, 'Loss:', 43.784197641848586, 'Test Loss:', 103.29953979119527)
('Epoch:', 7, 'Cluster #:', 25, 'Loss:', 39.676348549440156, 'Test Loss:', 94.9497262534533)
('Epoch:', 8, 'Cluster #:', 25, 'Loss:', 36.70157310644514, 'Test Loss:', 88.89629495775293)
('Epoch:', 9, 'Cluster #:', 25, 'Loss:', 33.68777084685018, 'Test Loss:', 85.53309237194794)
('Epoch:', 10, 'Cluster #:', 25, 'Loss:', 31.62626134102262, 'Test Loss:', 80.67354718668425)
('Epoch:', 11, 'Cluster #:', 25, 'Loss:', 30.100850980100514, 'Test Loss:', 83.60007793995133)
('Epoch:', 12, 'Cluster #:', 25, 'Loss:', 27.972241665369946, 'Test Loss:', 78.48380810162551)
('Epoch:', 13, 'Cluster #:', 25, 'Loss:', 26.79718875462626, 'Test Loss:', 78.7881210642461)
('Epoch:', 14, 'Cluster #:', 25, 'Loss:', 25.295018676088365, 'Test Loss:', 75.88227921965843)
('Epoch:', 15, 'Cluster #:', 25, 'Loss:', 24.44286949604233, 'Test Loss:', 71.73972092286799)
2018-12-10 02:39:45.787354

CRIM evaluation:
MRR: 0.02308
P@1: 0.01201
P@5: 0.01234
P@10: 0.01186

---------------------------------------------------------------
* ('m: ', 10, 'lambda: ', 0.1, 'max epoch per cluster: ', 20, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'sigmoid_constraint': 'ForceToOne', dropout=0.3, learning_rate=0.001)


('Sample clusters size: ', 11779)
('Epoch:', 1, 'Cluster #:', 30, 'Loss:', 164.64302213018138, 'Test Loss:', 16.821501479359963)
('Epoch:', 2, 'Cluster #:', 30, 'Loss:', 85.95771886678412, 'Test Loss:', 9.479251270126163)
('Epoch:', 3, 'Cluster #:', 30, 'Loss:', 60.89030767480532, 'Test Loss:', 7.113059009361313)
('Epoch:', 4, 'Cluster #:', 30, 'Loss:', 49.35616979487046, 'Test Loss:', 5.911477896573342)
('Epoch:', 5, 'Cluster #:', 30, 'Loss:', 42.839287023517926, 'Test Loss:', 5.237530594305887)
('Epoch:', 6, 'Cluster #:', 30, 'Loss:', 37.95000084499964, 'Test Loss:', 4.801821103322982)
('Epoch:', 7, 'Cluster #:', 30, 'Loss:', 34.69708706878203, 'Test Loss:', 4.445396709869783)
('Epoch:', 8, 'Cluster #:', 30, 'Loss:', 31.344791742943926, 'Test Loss:', 4.20229308469837)
('Epoch:', 9, 'Cluster #:', 30, 'Loss:', 29.270582812105324, 'Test Loss:', 4.001659726132235)
('Epoch:', 10, 'Cluster #:', 30, 'Loss:', 27.312983008645823, 'Test Loss:', 3.8331675905460845)
('Epoch:', 11, 'Cluster #:', 30, 'Loss:', 25.86424312723296, 'Test Loss:', 3.7695775713167223)
('Epoch:', 12, 'Cluster #:', 30, 'Loss:', 24.26373833860683, 'Test Loss:', 3.643787452736217)
('Epoch:', 13, 'Cluster #:', 30, 'Loss:', 23.194776870778636, 'Test Loss:', 3.5394367815238903)
('Epoch:', 14, 'Cluster #:', 30, 'Loss:', 22.077418524692256, 'Test Loss:', 3.4249304910639884)
('Epoch:', 15, 'Cluster #:', 30, 'Loss:', 20.919567144932383, 'Test Loss:', 3.3781916145887294)
('Epoch:', 16, 'Cluster #:', 30, 'Loss:', 20.137791508837896, 'Test Loss:', 3.305138163670991)
('Epoch:', 17, 'Cluster #:', 30, 'Loss:', 19.10677393638859, 'Test Loss:', 3.261422005968933)
('Epoch:', 18, 'Cluster #:', 30, 'Loss:', 18.686860465103926, 'Test Loss:', 3.1841310813365644)
('Epoch:', 19, 'Cluster #:', 30, 'Loss:', 17.963673188673297, 'Test Loss:', 3.117555515795205)
('Epoch:', 20, 'Cluster #:', 30, 'Loss:', 16.913953626791166, 'Test Loss:', 3.1116348788942028)
2018-12-10 12:28:42.204443

* Cluster distribution
Counter({0: 68,
         1: 516,
         2: 56,
         3: 1415,
         4: 345,
         5: 30,
         6: 452,
         7: 38,
         8: 45,
         9: 693,
         10: 2442,
         11: 18,
         12: 908,
         13: 28,
         14: 141,
         15: 1306,
         16: 84,
         17: 315,
         18: 412,
         19: 394,
         20: 364,
         21: 60,
         22: 189,
         23: 105,
         24: 24,
         25: 127,
         26: 731,
         27: 119,
         28: 34,
         29: 320})
         
CRIM evaluation:
MRR: 0.02787
P@1: 0.01935
P@5: 0.01447
P@10: 0.01298


* After fitting a KNN model on the cluster results and using it to fine-tune which clusters to compute prediction of each query term, the results improved three-fold.

CRIM evaluation:
MRR: 0.06088
P@1: 0.0447
P@5: 0.02825
P@10: 0.02616

* However, Yamane is, in general, disappointing with respect to SharedTask challenge.  

----------------------------------------------------------------------------------------
Training started...
('m: ', 5, 'lambda: ', 0.05, 'max epoch per cluster: ', 10, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'sigmoid_kernel_constraint: ', 'None', 'dropout: ', 0.2, 'learning_rate: ', 0.001, 'cluster_max: ', 15)
('Sample clusters size: ', 11779)
('Epoch:', 1, 'Cluster #:', 3, 'Loss:', 1109.3663462693028, 'Test Loss:', 50.47739016701659)
('Epoch:', 2, 'Cluster #:', 4, 'Loss:', 443.7235447903154, 'Test Loss:', 17.8407350068523)
('Epoch:', 3, 'Cluster #:', 4, 'Loss:', 337.1281598309441, 'Test Loss:', 20.06709446922047)
('Epoch:', 4, 'Cluster #:', 4, 'Loss:', 267.10441667238143, 'Test Loss:', 19.677748865275674)
('Epoch:', 5, 'Cluster #:', 4, 'Loss:', 229.65082203910208, 'Test Loss:', 17.97698300000083)
('Epoch:', 6, 'Cluster #:', 4, 'Loss:', 193.899041174633, 'Test Loss:', 16.751907205700498)
('Epoch:', 7, 'Cluster #:', 4, 'Loss:', 174.12937078346036, 'Test Loss:', 16.246821108061788)
('Epoch:', 8, 'Cluster #:', 5, 'Loss:', 143.5690400129045, 'Test Loss:', 8.259589462610172)
('Epoch:', 9, 'Cluster #:', 5, 'Loss:', 122.85453648527569, 'Test Loss:', 10.32649805369518)
('Epoch:', 10, 'Cluster #:', 5, 'Loss:', 120.9294774463863, 'Test Loss:', 11.019405025631883)
('Epoch:', 11, 'Cluster #:', 5, 'Loss:', 117.86586058200308, 'Test Loss:', 11.315035963446595)
('Epoch:', 12, 'Cluster #:', 5, 'Loss:', 117.66658123196073, 'Test Loss:', 11.45983252564142)
('Epoch:', 13, 'Cluster #:', 5, 'Loss:', 117.53339115441747, 'Test Loss:', 11.49364635983179)
('Epoch:', 14, 'Cluster #:', 5, 'Loss:', 117.17556676329085, 'Test Loss:', 11.589454222113588)
('Epoch:', 15, 'Cluster #:', 5, 'Loss:', 116.93114152015157, 'Test Loss:', 11.61919019261072)
('Epoch:', 16, 'Cluster #:', 5, 'Loss:', 116.64158589304395, 'Test Loss:', 11.655326009184815)
('Epoch:', 17, 'Cluster #:', 5, 'Loss:', 116.35099397958227, 'Test Loss:', 11.681808996588686)
2018-12-10 21:24:08.962568

CRIM evaluation:
MRR: 0.05966
P@1: 0.03669
P@5: 0.02672
P@10: 0.02536

* After predicting query term clusters
CRIM evaluation:
MRR: 0.06088
P@1: 0.0447
P@5: 0.02825
P@10: 0.02616

----------------------------------------------------------------------------------------------
('m: ', 5, 'lambda: ', 0.1, 'max epoch per cluster: ', 10, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'sigmoid_kernel_constraint: ', 'ForceToOne', 'dropout: ', 0.3, 'learning_rate: ', 0.001, 'cluster_max: ', 15)
('Sample clusters size: ', 11779)
('Epoch:', 1, 'Cluster #:', 9, 'Loss:', 482.0829993935509, 'Test Loss:', 27.62408774221937)
('Epoch:', 2, 'Cluster #:', 10, 'Loss:', 281.1839863856323, 'Test Loss:', 17.055058593617286)
('Epoch:', 3, 'Cluster #:', 11, 'Loss:', 208.62401276072805, 'Test Loss:', 12.390350534596523)
('Epoch:', 4, 'Cluster #:', 12, 'Loss:', 166.41537183626983, 'Test Loss:', 9.618624108000708)
('Epoch:', 5, 'Cluster #:', 13, 'Loss:', 134.7260290642522, 'Test Loss:', 7.7731040777703475)
('Epoch:', 6, 'Cluster #:', 13, 'Loss:', 119.45636204084552, 'Test Loss:', 7.063226612456044)
('Epoch:', 7, 'Cluster #:', 13, 'Loss:', 108.30129835629151, 'Test Loss:', 6.616330906035168)
('Epoch:', 8, 'Cluster #:', 13, 'Loss:', 101.58308503461907, 'Test Loss:', 6.293532006980128)
('Epoch:', 9, 'Cluster #:', 13, 'Loss:', 94.7640274246464, 'Test Loss:', 5.976787171151955)
('Epoch:', 10, 'Cluster #:', 13, 'Loss:', 88.485667872232, 'Test Loss:', 5.796518988622176)
('Epoch:', 11, 'Cluster #:', 13, 'Loss:', 88.32206717526479, 'Test Loss:', 5.7801972194946964)
('Epoch:', 12, 'Cluster #:', 13, 'Loss:', 88.03503130540048, 'Test Loss:', 5.762659813637024)
('Epoch:', 13, 'Cluster #:', 13, 'Loss:', 87.9104809983742, 'Test Loss:', 5.754546612789104)
('Epoch:', 14, 'Cluster #:', 13, 'Loss:', 87.80618179810531, 'Test Loss:', 5.747131267707188)
2018-12-11 00:15:28.930610

CRIM evaluation:
MRR: 0.05927
P@1: 0.03602
P@5: 0.03072
P@10: 0.02855

* After KNN

CRIM evaluation:
MRR: 0.06201
P@1: 0.03736
P@5: 0.03205
P@10: 0.02976

------------------------------------------------------------------------------------------------

('Sample clusters size: ', 11779)
('m: ', 1, 'lambda: ', 0.13, 'max epoch per cluster: ', 10, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_identity', 'sigmoid_kernel_constraint: ', 'ForceToOne', 'dropout: ', 0.3, 'learning_rate: ', 0.001, 'cluster_max: ', 25)
('Epoch:', 1, 'Cluster #:', 2, 'Loss:', 2540.3022823217325, 'Test Loss:', 53.87453193264082)
('Epoch:', 2, 'Cluster #:', 2, 'Loss:', 1667.1898109320173, 'Test Loss:', 40.18764992605429)
('Epoch:', 3, 'Cluster #:', 2, 'Loss:', 1361.9321219512858, 'Test Loss:', 34.321806932479376)
('Epoch:', 4, 'Cluster #:', 2, 'Loss:', 1178.6240238926068, 'Test Loss:', 30.980050023383228)
('Epoch:', 5, 'Cluster #:', 2, 'Loss:', 1044.863646138525, 'Test Loss:', 27.789724176647724)
('Epoch:', 6, 'Cluster #:', 3, 'Loss:', 643.6681745337323, 'Test Loss:', 16.18063761241986)
('Epoch:', 7, 'Cluster #:', 3, 'Loss:', 585.461576843336, 'Test Loss:', 14.75499623647435)
('Epoch:', 8, 'Cluster #:', 3, 'Loss:', 543.6171705845585, 'Test Loss:', 13.808908825924542)
('Epoch:', 9, 'Cluster #:', 3, 'Loss:', 507.70913083471424, 'Test Loss:', 12.98376774797604)
('Epoch:', 10, 'Cluster #:', 3, 'Loss:', 484.66481586770334, 'Test Loss:', 12.76265995549602)
('Epoch:', 11, 'Cluster #:', 3, 'Loss:', 483.03113291140517, 'Test Loss:', 12.732977458312538)
('Epoch:', 12, 'Cluster #:', 3, 'Loss:', 481.7263510373843, 'Test Loss:', 12.668654509856728)
('Epoch:', 13, 'Cluster #:', 3, 'Loss:', 481.51302522893866, 'Test Loss:', 12.680976091299877)
('Epoch:', 14, 'Cluster #:', 3, 'Loss:', 480.39035817460336, 'Test Loss:', 12.628896979962215)
('Epoch:', 15, 'Cluster #:', 3, 'Loss:', 481.034190520407, 'Test Loss:', 12.582702009354458)
2018-12-11 01:20:21.632953

CRIM evaluation:
MRR: 0.06717
MAP: 0.03322
P@1: 0.0427
P@5: 0.03244
P@10: 0.03148

----------------------------------------------------------------------------------------------
* Attempted different technique whereby the first (default) cluster is trained exclusively during the first epoch.  This follows the initialisation assumption that sets every training word pair to first cluster.
* The results were not impressive but better than observed with Yamane so far.
* Also, I'm limiting the number of clusters to 25, a ceiling which is reached immediately in the 2nd epoch.
* Must experiment with leaving clusters open (until no more are learnt); or set a much higher value.
* Alternatively we can set a threshold beyond which training examples are allocated to a "new" cluster but this cluster is not  created.  This still limits the model from growing uncontrollably;

Training started at: 2018-12-12 21:48:50.279247
('m: ', 10, 'lambda: ', 0.15, 'max epoch per cluster: ', 10, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'sigmoid_kernel_constraint: ', 'ForceToOne', 'dropout: ', 0.3, 'learning_rate: ', 0.001, 'cluster_max: ', 25)
('Sample clusters size: ', 11779)
Running evaluation on trial data set...
('Epoch:', 1, 'Cluster #:', 1, 'Loss:', 2645.7672815788537, 'Test MRR:', 0.10333)
coalition annotation
0 [[0.14232686]]
battersea_arts_centre constructed_structure
1 [[0.12914188]]
lyceum constructed_structure
1 [[0.14685696]]
('Processed ', 1000, 'samples...')
storyteller beguiler
1 [[0.14815803]]
paris center
2 [[0.146684]]
('Processed ', 2000, 'samples...')
union_station moving-picture_show
5 [[0.11579892]]
world wonder
1 [[0.11191753]]
('Processed ', 3000, 'samples...')
atmosphere magnitude_relation
5 [[0.13706593]]
true_life moving-picture_show
8 [[0.12733255]]
sight perspective
9 [[0.14652736]]
('Processed ', 4000, 'samples...')
young inebriant
8 [[0.14763217]]
('Processed ', 5000, 'samples...')
lucrezia_borgia moving-picture_show
8 [[0.11355487]]
tulsa moving-picture_show
12 [[0.08012623]]
('Processed ', 6000, 'samples...')
under_the_moon series
13 [[0.13277091]]
dale natural_depression
14 [[0.14759865]]
dale vale
15 [[0.1460675]]
('Processed ', 7000, 'samples...')
seaquake trouble
16 [[0.14310668]]
scott hamlet
14 [[0.13601676]]
('Processed ', 8000, 'samples...')
hamlet moving-picture_show
18 [[0.11416043]]
sight moving-picture_show
19 [[0.08516396]]
('Processed ', 9000, 'samples...')
power probality
19 [[0.1498438]]
touch_and_go moving-picture_show
21 [[0.14001596]]
('Processed ', 10000, 'samples...')
star separable_space
20 [[0.1300958]]
scott mathematical_relation
23 [[0.12929982]]
('Processed ', 11000, 'samples...')
('Epoch:', 2, 'Cluster #:', 25, 'Loss:', 156.81297644650564, 'Test MRR:', 0.05333)
('Epoch:', 3, 'Cluster #:', 25, 'Loss:', 87.73115104873665, 'Test MRR:', 0.06417)
('Epoch:', 4, 'Cluster #:', 25, 'Loss:', 73.36540189261082, 'Test MRR:', 0.05733)
('Epoch:', 5, 'Cluster #:', 25, 'Loss:', 64.38504860263784, 'Test MRR:', 0.059)
('Epoch:', 6, 'Cluster #:', 25, 'Loss:', 56.81815980266779, 'Test MRR:', 0.078)
('Epoch:', 7, 'Cluster #:', 25, 'Loss:', 51.290908942183016, 'Test MRR:', 0.094)
('Epoch:', 8, 'Cluster #:', 25, 'Loss:', 47.23172779464861, 'Test MRR:', 0.08517)
('Epoch:', 9, 'Cluster #:', 25, 'Loss:', 43.46400957543636, 'Test MRR:', 0.08822)
('Epoch:', 10, 'Cluster #:', 25, 'Loss:', 40.14030005659443, 'Test MRR:', 0.07756)
('Epoch:', 11, 'Cluster #:', 25, 'Loss:', 38.91978675389197, 'Test MRR:', 0.06556)
Training concluded at: 2018-12-12 23:16:56.594990

CRIM evaluation:
MRR: 0.09636
MAP: 0.04405
P@1: 0.06871
P@5: 0.04286
P@10: 0.04008

* After clustering test queries using KNN: (slight improvement)

CRIM evaluation:
MRR: 0.09942
MAP: 0.04663
P@1: 0.07071
P@5: 0.04554
P@10: 0.04292

----------------------------------------------------------------------------------------------------
* Loosened ceiling somewhat to create 30 clusters and reduced the lambda threshold to 0.13 (from 0.15).  Clusters still increased to 30 pretty quickly.  

Training started at: 2018-12-13 00:32:10.229728
('m: ', 10, 'lambda: ', 0.13, 'max epoch per cluster: ', 10, 'Negative sampling: ', 'random', 'Phi Init: ', 'random_plus_identity', 'sigmoid_kernel_constraint: ', 'ForceToOne', 'dropout: ', 0.3, 'learning_rate: ', 0.001, 'cluster_max: ', 30)
('Sample clusters size: ', 11779)
('Epoch:', 1, 'Cluster #:', 1, 'Loss:', 2648.454608650878, 'Test MRR:', 0.10333)
coalition annotation
0 [[0.11573532]]
country popular_music_genre
1 [[0.11753921]]
campo_san_polo hausdorff_space
1 [[0.12842865]]
('Processed ', 1000, 'samples...')
paris series
2 [[0.11816365]]
('Processed ', 2000, 'samples...')
grand moving-picture_show
4 [[0.08930109]]
sight season
4 [[0.1250305]]
('Processed ', 3000, 'samples...')
rotunda constructed_structure
5 [[0.12891144]]
true_life moving-picture_show
6 [[0.10975606]]
('Processed ', 4000, 'samples...')
century si_base_unit
7 [[0.12408465]]
telemovie data-storage_medium
7 [[0.12526542]]
('Processed ', 5000, 'samples...')
lucrezia_borgia moving-picture_show
10 [[0.11526864]]
tulsa moving-picture_show
11 [[0.08711283]]
('Processed ', 6000, 'samples...')
nancy sailing_boat
5 [[0.12192791]]
dale vale
13 [[0.11943772]]
('Processed ', 7000, 'samples...')
seaquake trouble
14 [[0.12455411]]
scott hamlet
15 [[0.12320519]]
('Processed ', 8000, 'samples...')
hamlet moving-picture_show
16 [[0.10422902]]
sight moving-picture_show
17 [[0.07907726]]
('Processed ', 9000, 'samples...')
campo_san_polo topological_manifold
18 [[0.09345377]]
('Processed ', 10000, 'samples...')
star separable_space
16 [[0.10601153]]
scott mathematical_relation
20 [[0.12851807]]
('Processed ', 11000, 'samples...')
knickers physiological_property
21 [[0.11303463]]
new_orleans moving-picture_show
21 [[0.08724869]]
Running evaluation on trial data set...
('Epoch:', 2, 'Cluster #:', 24, 'Loss:', 160.69473196558343, 'Test MRR:', 0.07)
campo_san_polo hausdorff_space
21 [[0.11840388]]
sight view
19 [[0.12602326]]
('Processed ', 1000, 'samples...')
lyceum constructed_structure
1 [[0.12324508]]
('Processed ', 2000, 'samples...')
hotel_europe constructed_structure
26 [[0.12173095]]
campo_san_polo way
27 [[0.09194536]]
('Processed ', 3000, 'samples...')
keshu moving-picture_show
28 [[0.11239138]]
('Epoch:', 3, 'Cluster #:', 30, 'Loss:', 88.85481828417979, 'Test MRR:', 0.07286)
('Epoch:', 4, 'Cluster #:', 30, 'Loss:', 64.39664095826059, 'Test MRR:', 0.07467)
('Epoch:', 5, 'Cluster #:', 30, 'Loss:', 56.075072061417934, 'Test MRR:', 0.09)
('Epoch:', 6, 'Cluster #:', 30, 'Loss:', 49.8601350386045, 'Test MRR:', 0.11167)
('Epoch:', 7, 'Cluster #:', 30, 'Loss:', 45.05676133218609, 'Test MRR:', 0.11622)
('Epoch:', 8, 'Cluster #:', 30, 'Loss:', 41.27733868958215, 'Test MRR:', 0.09743)
('Epoch:', 9, 'Cluster #:', 30, 'Loss:', 37.57391656068697, 'Test MRR:', 0.08908)
('Epoch:', 10, 'Cluster #:', 30, 'Loss:', 34.95721684883465, 'Test MRR:', 0.08286)
('Epoch:', 11, 'Cluster #:', 30, 'Loss:', 33.32479047970652, 'Test MRR:', 0.08286)
('Epoch:', 12, 'Cluster #:', 30, 'Loss:', 33.07274405640977, 'Test MRR:', 0.08286)
Training concluded at: 2018-12-13 02:46:15.895080

CRIM evaluation:
MRR: 0.10991
MAP: 0.05083
P@1: 0.07805
P@5: 0.04998
P@10: 0.04715

* After KNN
CRIM evaluation:
MRR: 0.10861
MAP: 0.04981
P@1: 0.08005
P@5: 0.04831
P@10: 0.04565



# Scratch Pad

In [ ]:
model.vocab['dog'].index


In [ ]:
test_these_terms = np.asarray(data.tokenizer.texts_to_sequences(data.train_query))
test_these_hypers = np.asarray(data.tokenizer.texts_to_sequences(data.train_hyper))

indices = np.arange(len(test_these_terms))                               
#np.random.seed(32)
np.random.shuffle(indices)
print indices[:32]

term, hyper, label = extend_batch_with_negatives(test_these_terms[indices[:32]], test_these_hypers[indices[:32]], data.random_words, data.tokenizer, 5)
print len(term)
[(data.tokenizer.index_word[i[0]], data.tokenizer.index_word[j[0]], l) for i, j, l in zip(term, hyper, label)]

In [ ]:
#hyper_candidates = [[data.tokenizer.word_index[hyper]] for hyper in data.vocab]
len(set(data.valid_hyper))

In [ ]:

[l for l in crim_model.layers if type(l) == Dense][0].get_weights()[0]

In [ ]:
np.sort(data.tokenizer.texts_to_sequences(data.vocab)).shape



In [38]:
#for idx, m in enumerate(clusters):
 #   print idx, m.loss, m.test_loss, m.model.get_layer(name='Prediction').get_weights()[0]

i = data.tokenizer.word_index['vegeterian']
j = data.tokenizer.word_index['person']
for c in clusters:
    print c.model.predict([[i], [j]])[0]

[0.8394212]
[0.19308254]
[0.11467936]
[0.15988037]
[0.10992873]
[0.11244643]
[0.16883272]
[0.09364048]
[0.10421132]
[0.11453836]


In [ ]:
len(data.test_query)

In [ ]:
def get_cluster_CRIM_model(phi_k=1,
                           embeddings_layer=None,
                           embeddings_dim = 200,
                           phi_init = None,
                           phi_activity_regularisation = None,
                           sigmoid_kernel_regularisation = None,
                           sigmoid_bias_regularisation = None,
                           sigmoid_kernel_constraint = None,
                           dropout_rate = 0.,
                           learning_rate = 0.001
                  ):
    
    hypo_input  = Input(shape=(1,), name='Hyponym')
    hyper_input = Input(shape=(1,), name='Hypernym')
        
    hypo_embedding, hyper_embedding = embeddings_layer([hypo_input, hyper_input])
            
    # Add Dropout to avoid overfit    
    hypo_embedding = Dropout(dropout_rate, name='Dropout_Hypo')(hypo_embedding)
    hyper_embedding = Dropout(dropout_rate, name='Dropout_Hyper')(hyper_embedding)
    
    phi_layer = []
    for i in range(phi_k):
        phi_layer.append(Dense(embeddings_dim, activation=None, use_bias=False, 
                               activity_regularizer=phi_activity_regularisation,
                               kernel_initializer=phi_init,                               
                               name='Phi%d' % (i)) (hypo_embedding))            

    if phi_k == 1:
        # flatten tensors
        phi = Flatten()(phi_layer[0])
        hyper_embedding = Flatten()(hyper_embedding)    
    else:
        phi = concatenate(phi_layer, axis=1)
    
    phi = Dropout(dropout_rate, name='Dropout_Phi')(phi)
    
    # this is referred to as "s" in the "CRIM" paper    
    phi_hyper = Dot(axes=-1, normalize=True, name='DotProduct')([phi, hyper_embedding])
    
    if phi_k > 1:
        phi_hyper = Flatten()(phi_hyper)
    
    predictions = Dense(1, activation="sigmoid", name='Prediction',
                        use_bias=True,
                        kernel_initializer='random_normal',
                        kernel_constraint= sigmoid_kernel_constraint,
                        bias_initializer='random_normal',                        
                        kernel_regularizer=sigmoid_kernel_regularisation,
                        bias_regularizer=sigmoid_bias_regularisation
                       ) (phi_hyper)

    # instantiate model
    model = Model(inputs=[hypo_input, hyper_input], outputs=predictions)        

    adam = Adam(lr = learning_rate, beta_1 = 0.9, beta_2 = 0.9, clipnorm=1.)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
import gc
del clusters

gc.collect()


In [41]:
from collections import Counter
Counter(np.random.choice(np.arange(len(data.train_query)), size=11778, replace=True))

Counter({2: 1,
         3: 1,
         4: 3,
         5: 1,
         6: 3,
         8: 1,
         12: 1,
         13: 1,
         14: 2,
         15: 2,
         16: 2,
         19: 2,
         20: 1,
         21: 1,
         25: 1,
         26: 1,
         28: 2,
         29: 1,
         31: 3,
         32: 2,
         33: 2,
         35: 2,
         36: 1,
         38: 2,
         40: 1,
         41: 1,
         42: 1,
         47: 2,
         48: 1,
         50: 2,
         51: 2,
         54: 2,
         56: 2,
         59: 3,
         60: 3,
         61: 1,
         67: 1,
         69: 3,
         70: 2,
         73: 1,
         74: 1,
         75: 1,
         77: 1,
         79: 2,
         80: 2,
         81: 1,
         82: 1,
         85: 1,
         88: 1,
         90: 2,
         92: 1,
         93: 2,
         94: 1,
         96: 1,
         98: 1,
         99: 3,
         100: 1,
         102: 1,
         103: 1,
         104: 1,
         106: 2,
         107: 3,
        

In [27]:
embeddings_layer = get_embeddings_model(embedding_matrix=data.embedding_matrix)
test_crim_model = get_CRIM_model(phi_k = 1, train_embeddings = True,
                            embeddings_dim = data.embeddings_dim, 
                            #vocab_size = len(data.tokenizer.word_counts), embedding = data.embedding_matrix,
                            embeddings_layer = embeddings_layer,
                            phi_init = RandomIdentity(),                            
                            sigmoid_kernel_regularisation = None, #l2(0.001),
                            sigmoid_bias_regularisation = None, #l2(0.001),
                            sigmoid_kernel_constraint = None,
                            dropout_rate = 0.3,
                            learning_rate = 0.0001
                           )

In [28]:
test_crim_model.get_layer(name='Phi0').get_weights()[0]

array([[-0.00319363, -0.        , -0.        , ..., -0.        ,
        -0.        ,  0.        ],
       [-0.        ,  0.00204002,  0.        , ..., -0.        ,
        -0.        ,  0.        ],
       [ 0.        , -0.        , -0.00759161, ...,  0.        ,
        -0.        , -0.        ],
       ...,
       [-0.        ,  0.        ,  0.        , ..., -0.01416957,
         0.        ,  0.        ],
       [-0.        , -0.        , -0.        , ..., -0.        ,
         0.01755028,  0.        ],
       [ 0.        , -0.        , -0.        , ..., -0.        ,
         0.        , -0.00710576]], dtype=float32)

In [29]:
test_crim_model_2 = get_CRIM_model_freeze_phi(test_crim_model,
                                             train_phi = False,
                                             train_embeddings = True,
                                             dropout_rate = 0.3,
                                             learning_rate = 0.0001)



In [30]:
test_crim_model_3 = get_CRIM_model_freeze_phi(test_crim_model_2,
                                             train_phi = False,
                                             train_embeddings = True,
                                             dropout_rate = 0.3,
                                             learning_rate = 0.0001)

In [98]:
#cluster_list[0].model.get_layer(name='Phi0').get_weights()[0]

for idx, m in enumerate(clusters):
    print idx, np.mean(m.avg_loss), m.model.get_layer(name='Prediction').get_weights()[1]

i = data.tokenizer.word_index['dirham']
j = data.tokenizer.word_index['monetary_unit']
for c in clusters:
    print c.model.predict([[i], [j]])[0]
    


0 0.30369790482193465 [-2.2947042]
[0.1097306]


In [71]:
data.valid_query[0], data.valid_hyper[0]
i = data.tokenizer.word_index['dirham']
j = data.tokenizer.word_index['monetary_unit']


np.dot(data.embedding_matrix[i], data.embedding_matrix[j])



0.40714827

In [29]:
data.tokenizer.texts_to_sequences(data.valid_query[:4])

[[2922], [2089], [2089], [2089]]

In [59]:
filter(lambda x: x in data.test_query, data.test_hyper)

[u'bank',
 u'actor',
 u'spending',
 u'animation',
 u'judge',
 u'minister',
 u'island',
 u'response',
 u'minister',
 u'killer',
 u'horse',
 u'transportation',
 u'form',
 u'record_label',
 u'label',
 u'mark',
 u'watercraft',
 u'management',
 u'language',
 u'engineering',
 u'university',
 u'shopping',
 u'mark',
 u'discharge',
 u'noble',
 u'prefix',
 u'ideal',
 u'horse',
 u'peasant',
 u'maltreatment',
 u'hurt',
 u'persecution',
 u'actor',
 u'cricketer',
 u'standard',
 u'housing',
 u'offense',
 u'arm',
 u'basin',
 u'picture',
 u'bridge',
 u'proposal',
 u'form',
 u'sport',
 u'cap',
 u'island',
 u'mark',
 u'ideal',
 u'flag',
 u'prefix',
 u'language',
 u'offense',
 u'photographer',
 u'maltreatment',
 u'infrastructure',
 u'sport',
 u'policy',
 u'note',
 u'university',
 u'happening',
 u'university',
 u'note',
 u'sport',
 u'mark',
 u'net',
 u'teacher',
 u'horse',
 u'picture',
 u'magazine',
 u'local_government',
 u'spending',
 u'reflex_response',
 u'mark',
 u'theme',
 u'emptiness',
 u'watercraft',

In [4]:

a = np.asarray([10,5,3,13])
b = np.asarray([13,4,5,10])

(b - a) ** 2


array([9, 1, 4, 9])

In [29]:
print np.mean((model['tennis_player'] - model['rod_laver'])) 

print np.mean((model['athlete'] - model['rod_laver']) )

0.004040569
0.005828381
